# Soil AI — Colab Training Notebook
This notebook trains two baseline models from your cleaned dataset:
- **Atterberg → Soil Classification** (RandomForestClassifier)
- **CBR → Adopted CBR %** (RandomForestRegressor)

> **How to use:**  
> 1. Run the **Setup** cell.  
> 2. In **Load Data**, either **mount Google Drive** (if your CSV is in Drive) or **upload `soil_master_with_sources.csv`** from your computer.  
> 3. Run the training cells.  
> 4. Use the **Gradio App** cell for interactive predictions.


In [ ]:
# === Setup ===
# If running on Colab, you may install packages. On Colab these are typically preinstalled.
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    try:
        # scikit-learn and joblib are usually available
        import sklearn, joblib
    except Exception:
        !pip -q install scikit-learn joblib

    try:
        import gradio as gr
    except Exception:
        !pip -q install gradio

print("Colab environment:", IN_COLAB)


Colab environment: True


In [ ]:
# === Load Data ===
import pandas as pd, numpy as np, os

MASTER = None

if 'google.colab' in str(get_ipython()):  # If running in Colab
    from google.colab import drive, files
    print("Option A) Mount Drive (if your CSV is in Drive).")
    try:
        drive.mount('/content/drive', force_remount=True)
        # 👇 Update this path if your CSV lives elsewhere in Drive
        default_drive_path = '/content/drive/MyDrive/soil_master_with_sources.csv'
        if os.path.exists(default_drive_path):
            MASTER = default_drive_path
            print("Found master at:", MASTER)
    except Exception as e:
        print("Drive mount skipped:", e)

    if MASTER is None:
        print("Option B) Upload the CSV manually (choose your file).")
        uploaded = files.upload()  # pick soil_master_with_sources.csv
        MASTER = list(uploaded.keys())[0]
else:
    # Local/Jupyter fallback: look in the working directory
    if os.path.exists('soil_master_with_sources.csv'):
        MASTER = 'soil_master_with_sources.csv'
    else:
        raise FileNotFoundError("Please place 'soil_master_with_sources.csv' in the working directory.")

dfm = pd.read_csv(MASTER)
print('Rows:', len(dfm), '| Columns:', len(dfm.columns))
dfm.head(10)


Option A) Mount Drive (if your CSV is in Drive).
Mounted at /content/drive
Option B) Upload the CSV manually (choose your file).


In [ ]:
# === (A) Atterberg -> Soil Classification ===
import json, joblib
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

att = dfm[dfm['Test_Type']=='Atterberg'].copy()
att_pivot = att.pivot_table(index='SampleID', columns='Parameter', values='Value', aggfunc='first').reset_index()

for col in ['Liquid_Limit_LL(%)','Plastic_Limit_PL(%)','Plasticity_Index_PI(%)']:
    if col in att_pivot.columns:
        att_pivot[col] = pd.to_numeric(att_pivot[col], errors='coerce')

target_col = 'Soil_Classification'
if target_col not in att_pivot.columns:
    raise ValueError('No Soil_Classification labels found in your Atterberg data.')

att_pivot = att_pivot.dropna(subset=[target_col])
X_att = att_pivot[['Liquid_Limit_LL(%)','Plastic_Limit_PL(%)','Plasticity_Index_PI(%)']].copy()
y_att = att_pivot[target_col].astype(str)

mask_ok = X_att.notna().all(axis=1)
X_att, y_att = X_att[mask_ok], y_att[mask_ok]

le = LabelEncoder()
y_enc = le.fit_transform(y_att)

clf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
if len(X_att) >= 6 and len(np.unique(y_enc)) > 1:
    Xtr, Xva, ytr, yva = train_test_split(X_att, y_enc, test_size=0.3, random_state=42, stratify=y_enc)
    clf.fit(Xtr, ytr)
    ypred = clf.predict(Xva)
    print('Accuracy (holdout):', round(accuracy_score(yva, ypred), 3))
    print(classification_report(yva, ypred, target_names=le.classes_, zero_division=0))
else:
    clf.fit(X_att, y_enc)
    if len(X_att) >= 3:
        k = min(3, len(X_att))
        cv = cross_val_score(clf, X_att, y_enc, cv=k, scoring='accuracy')
        print('CV accuracy (k=%d):' % k, [round(v,3) for v in cv])
    else:
        print('Fitted on all data (too few samples for CV).')

joblib.dump(clf, 'atterberg_classifier_rf.pkl')
joblib.dump(le, 'atterberg_label_encoder.pkl')
print('Saved: atterberg_classifier_rf.pkl, atterberg_label_encoder.pkl')


In [ ]:
# === (B) CBR -> Adopted CBR % ===
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score

cbr = dfm[dfm['Test_Type']=='CBR'].copy()
cbr_pivot = cbr.pivot_table(index='SampleID', columns='Parameter', values='Value', aggfunc='first').reset_index()

for col in ['MDD_gcc','OMC_pct','Blows','Adopted_CBR_pct','CBR_2p5_pct','CBR_5p0_pct']:
    if col in cbr_pivot.columns:
        cbr_pivot[col] = pd.to_numeric(cbr_pivot[col], errors='coerce')

target = 'Adopted_CBR_pct' if 'Adopted_CBR_pct' in cbr_pivot.columns else 'CBR_5p0_pct'
features = [c for c in ['MDD_gcc','OMC_pct','Blows','CBR_2p5_pct'] if c in cbr_pivot.columns]

cbr_pivot = cbr_pivot.dropna(subset=[target])
X = cbr_pivot[features].copy().fillna(cbr_pivot[features].mean())
y = cbr_pivot[target].copy()

regr = RandomForestRegressor(n_estimators=300, random_state=42)
if len(X) >= 5:
    from sklearn.model_selection import train_test_split
    Xtr, Xva, ytr, yva = train_test_split(X, y, test_size=0.3, random_state=42)
    regr.fit(Xtr, ytr)
    ypred = regr.predict(Xva)
    print('MAE (holdout):', round(mean_absolute_error(yva, ypred), 3))
    print('R^2  (holdout):', round(r2_score(yva, ypred), 3))
else:
    regr.fit(X, y)
    if len(X) >= 3:
        k = min(3, len(X))
        mae = -cross_val_score(regr, X, y, cv=k, scoring='neg_mean_absolute_error')
        print('CV MAE (k=%d):' % k, [round(v,3) for v in mae])
    else:
        print('Fitted on all data (too few samples for CV).')

import joblib, json
joblib.dump(regr, 'cbr_regressor_rf.pkl')
with open('cbr_regressor_features.json','w') as f:
    json.dump(features, f)
print('Saved: cbr_regressor_rf.pkl, cbr_regressor_features.json')


In [ ]:
# === (C) Gradio App: Try the models ===
import gradio as gr, joblib, numpy as np, json

# Load models (ensure you've run training cells)
clf = joblib.load('atterberg_classifier_rf.pkl')
le  = joblib.load('atterberg_label_encoder.pkl')
regr = joblib.load('cbr_regressor_rf.pkl')
features = json.load(open('cbr_regressor_features.json'))

def predict_soil_class(ll, pl, pi):
    x = np.array([[ll, pl, pi]], dtype=float)
    pred = clf.predict(x)[0]
    return le.inverse_transform([pred])[0]

def predict_cbr(mdd, omc, blows, cbr25):
    # Build feature vector in saved order
    data = []
    for f in features:
        if f == 'MDD_gcc': data.append(mdd)
        elif f == 'OMC_pct': data.append(omc)
        elif f == 'Blows': data.append(blows)
        elif f == 'CBR_2p5_pct': data.append(cbr25 if cbr25 is not None else np.nan)
        else: data.append(np.nan)
    vec = np.array([np.nan if v=='' else v for v in data], dtype=float)
    # Simple imputation for any NaN
    vec = np.where(np.isnan(vec), np.nanmean(vec.astype(float)), vec)
    y = regr.predict([vec])[0]
    return float(np.round(y, 2))

with gr.Blocks() as demo:
    gr.Markdown("## Soil AI — Quick Predictions")
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Atterberg → Soil Class")
            ll = gr.Number(label="Liquid Limit (LL) %", value=32.0)
            pl = gr.Number(label="Plastic Limit (PL) %", value=22.0)
            pi = gr.Number(label="Plasticity Index (PI) %", value=10.0)
            btn1 = gr.Button("Predict Class")
            out1 = gr.Textbox(label="Predicted Class")
            btn1.click(fn=predict_soil_class, inputs=[ll, pl, pi], outputs=out1)
        with gr.Column():
            gr.Markdown("### CBR → Adopted CBR %")
            mdd = gr.Number(label="MDD (g/cc)", value=2.10)
            omc = gr.Number(label="OMC (%)", value=8.0)
            blows = gr.Number(label="Blows (#)", value=30)
            cbr25 = gr.Number(label="CBR at 2.5 mm (%) — optional", value=None)
            btn2 = gr.Button("Predict CBR")
            out2 = gr.Number(label="Predicted Adopted CBR (%)")
            btn2.click(fn=predict_cbr, inputs=[mdd, omc, blows, cbr25], outputs=out2)

demo


In [ ]:
import pandas as pd

MASTER = "/content/Soil_Master_with_Sources__Preview__....csv"  # put your exact file name here


In [ ]:
dfm = pd.read_csv(MASTER)

print("Rows:", len(dfm), "| Columns:", len(dfm.columns))
dfm.head(10)  # show the first 10 rows


FileNotFoundError: [Errno 2] No such file or directory: '/content/Soil_Master_with_Sources__Preview__....csv'

In [ ]:
import pandas as pd

MASTER = "/content/soil_master_with_sources.csv"

dfm = pd.read_csv(MASTER)
print("Rows:", len(dfm), "| Columns:", len(dfm.columns))
dfm.head(10)


Rows: 181 | Columns: 9


,BatchID,Test_Type,SampleID,Soil_Type,Parameter,Value,Units,Source,Notes
0,38,Sieve,Sample_3pdf,NaN,D10(mm),0.3510775862068965,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
1,39,Sieve,Sample_5pdf,NaN,D10(mm),NaN,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
2,41,Sieve,Sample_6pdf_Sand,NaN,D10(mm),0.2134615384615384,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
3,40,Sieve,Sample_6pdf_Granite,NaN,D10(mm),NaN,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
4,38,Sieve,Sample_3pdf,NaN,D30(mm),0.8105269645608628,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
5,39,Sieve,Sample_5pdf,NaN,D30(mm),NaN,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
6,41,Sieve,Sample_6pdf_Sand,NaN,D30(mm),0.425,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
7,40,Sieve,Sample_6pdf_Granite,NaN,D30(mm),NaN,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
8,38,Sieve,Sample_3pdf,NaN,D60(mm),1.9382449246889328,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
9,39,Sieve,Sample_5pdf,NaN,D60(mm),NaN,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)


In [ ]:
import numpy as np
import pandas as pd

# Start from the stacked master already loaded as dfm
wide = dfm.pivot_table(index=["SampleID"], columns="Parameter", values="Value", aggfunc="first").reset_index()

# Coerce numerics we need
num_cols = [
    "Percent_Passing_at_0.075mm",  # #200 (0.075 mm)
    "Percent_Passing_at_0.425mm",  # #40 (0.425 mm)
    "Percent_Passing_at_2.0mm",    # #10 (2.0 mm)
    "Cu","Cc",
    "Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)"
]
for c in num_cols:
    if c in wide.columns:
        wide[c] = pd.to_numeric(wide[c], errors="coerce")

# --- Helpers ---
def uscs_from(w):
    F = w.get("Percent_Passing_at_0.075mm")
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    Cu = w.get("Cu"); Cc = w.get("Cc")
    if pd.isna(F): return None

    # Coarse vs fine
    if F < 50:
        # Coarse-grained
        # Decide sand vs gravel using 4.75 mm (#4). If not present, we’ll just say “S or G”.
        # Clean vs with fines
        fines_class = None
        if F < 5: fines_class = "clean"
        elif F > 12: fines_class = "fines"
        else: fines_class = "dual"

        # Well/poorly graded (needs Cu & Cc)
        grad = None
        if pd.notna(Cu) and pd.notna(Cc):
            # Use sand criteria (similar for gravels): Cu>6 & 1<Cc<3 (for sands Cu>6, gravels Cu>4)
            # We’ll be conservative and use: Cu>6 and 1<Cc<3 => well graded (W), else poorly graded (P)
            grad = "W" if (Cu>6 and 1<Cc<3) else "P"

        if fines_class == "clean":
            return f"S{grad}" if grad else "S"
        elif fines_class == "fines":
            if pd.isna(LL) or pd.isna(PI): return "S(M/C)"
            # A-line: PI >= 0.73*(LL-20) ⇒ clayey (C), else silty (M)
            a_line = 0.73*(LL-20)
            return "SC" if PI >= a_line else "SM"
        else:
            # 5–12% fines → dual symbols; we’ll return generic
            return "SW-SM" if grad=="W" else "SP-SM"
    else:
        # Fine-grained
        if pd.isna(LL) or pd.isna(PI): return None
        a_line = 0.73*(LL-20)
        if LL < 50:
            return "CL" if PI >= a_line else "ML"
        else:
            return "CH" if PI >= a_line else "MH"

def aashto_gi_from(w):
    # AASHTO GI = (F−35)[0.2 + 0.005(LL−40)] + 0.01(F−15)(PI−10)
    F  = w.get("Percent_Passing_at_0.075mm")   # #200
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    if any(pd.isna(x) for x in [F,LL,PI]): return None
    GI = (max(F-35,0))*(0.2 + 0.005*max(LL-40,0)) + 0.01*max(F-15,0)*max(PI-10,0)
    return round(float(GI),2)

wide["USCS_rule"] = wide.apply(uscs_from, axis=1)
wide["AASHTO_GI"] = wide.apply(aashto_gi_from, axis=1)

# Keep existing reported class if you had it; otherwise backfill with rule
if "Soil_Classification" in wide.columns:
    wide["Soil_Class_final"] = wide["Soil_Classification"]
    wide.loc[wide["Soil_Class_final"].isna() | (wide["Soil_Class_final"]==""), "Soil_Class_final"] = wide["USCS_rule"]
else:
    wide["Soil_Class_final"] = wide["USCS_rule"]

print(wide[["SampleID","USCS_rule","AASHTO_GI","Soil_Class_final"]].head(10))
wide.to_csv("labeled.csv", index=False)
print("Saved labeled.csv with rule-based USCS/AASHTO columns.")


Parameter                   SampleID USCS_rule AASHTO_GI  \
0            Atterberg_CSU_LongBeach      None      None   
1              Atterberg_Cyprus_Intl      None      None   
2                Atterberg_Indonesia      None      None   
3            Atterberg_Namibia_GTM7b      None      None   
4              Atterberg_UiTM_CEG454      None      None   
5             Atterberg_UiTM_ConePen      None      None   
6          Atterberg_UiTM_FullReport      None      None   
7           Atterberg_UiTM_Pahang_PL      None      None   
8            Atterberg_UiTM_ShahAlam      None      None   
9           Atterberg_UiTM_ShahAlam2      None      None   

Parameter          Soil_Class_final  
0                      Plastic Clay  
1                Non-plastic / Silt  
2            Clay (High Plasticity)  
3                              Clay  
4                 Intermediate Clay  
5          CLAY with low plasticity  
6                              None  
7             Clay (Low Plasticity)

In [ ]:
# List columns we can use
print("All columns:", dfm.columns.tolist())

# What 'Percent_Passing_at_*' parameters exist after pivot?
wide_dbg = dfm.pivot_table(index="SampleID", columns="Parameter", values="Value", aggfunc="first").reset_index()
pp_cols = [c for c in wide_dbg.columns if str(c).startswith("Percent_Passing_at_")]
print("Percent_Passing_at_* columns:", pp_cols[:20])


All columns: ['BatchID', 'Test_Type', 'SampleID', 'Soil_Type', 'Parameter', 'Value', 'Units', 'Source', 'Notes']
Percent_Passing_at_* columns: []


In [ ]:
import re
import numpy as np
import pandas as pd

# 1) Build a wide table from the stacked master dfm
wide = dfm.pivot_table(index=["SampleID"], columns="Parameter", values="Value", aggfunc="first").reset_index()

# 2) Coerce numerics we may use
for c in ["Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)","Cu","Cc",
          "%Fines","%Sand","%Gravel"]:
    if c in wide.columns:
        wide[c] = pd.to_numeric(wide[c], errors="coerce")

# 3) Try to find %Passing for #200 (0.075mm), #40 (0.425mm), #10 (2.0mm) if they exist,
#    otherwise backfill F200 from %Fines (typical equivalence).
pp_cols = [c for c in wide.columns if str(c).startswith("Percent_Passing_at_")]

def pick_mm(colnames, target_mm, tol):
    # choose column whose embedded size is closest to target within tolerance
    best_col, best_d = None, 1e9
    for c in colnames:
        m = re.search(r"Percent_Passing_at_([0-9.]+)\s*mm", str(c))
        if not m:
            continue
        try:
            size = float(m.group(1))
        except:
            continue
        d = abs(size - target_mm)
        if d < best_d:
            best_col, best_d = c, d
    return best_col if best_d <= tol else None

col_F200 = pick_mm(pp_cols, 0.075, tol=0.02)  # No.200
col_F40  = pick_mm(pp_cols, 0.425, tol=0.05)  # No.40
col_F10  = pick_mm(pp_cols, 2.0,   tol=0.20)  # No.10

# Create F200/F40/F10 numeric columns if found
if col_F200: wide["F200"] = pd.to_numeric(wide[col_F200], errors="coerce")
if col_F40:  wide["F40"]  = pd.to_numeric(wide[col_F40],  errors="coerce")
if col_F10:  wide["F10"]  = pd.to_numeric(wide[col_F10],  errors="coerce")

# Backfill F200 from %Fines if needed
if "F200" not in wide.columns and "%Fines" in wide.columns:
    wide["F200"] = wide["%Fines"]

# 4) USCS & AASHTO rule functions
def a_line(LL):
    return 0.73*(LL-20)

def uscs_from_row(r):
    F  = r.get("F200")
    LL = r.get("Liquid_Limit_LL(%)")
    PI = r.get("Plasticity_Index_PI(%)")
    Cu = r.get("Cu")
    Cc = r.get("Cc")
    if pd.isna(F):
        return None

    if F < 50:  # coarse-grained (S/G) — we only have sand/grading info, so default to S*
        if F < 5:
            if pd.notna(Cu) and pd.notna(Cc):
                return "SW" if (Cu>6 and 1<Cc<3) else "SP"
            return "S"   # clean sand (grading unknown)
        elif F > 12:
            if pd.isna(LL) or pd.isna(PI):
                return "S?(with fines)"
            return "SC" if PI >= a_line(LL) else "SM"
        else:  # 5–12% fines → dual symbols
            if pd.notna(Cu) and pd.notna(Cc) and (Cu>6 and 1<Cc<3):
                return "SW-SM"
            return "SP-SM"
    else:      # fine-grained
        if pd.isna(LL) or pd.isna(PI):
            return None
        return ("CL" if PI >= a_line(LL) else "ML") if LL < 50 else ("CH" if PI >= a_line(LL) else "MH")

def aashto_gi(r):
    F  = r.get("F200")
    LL = r.get("Liquid_Limit_LL(%)")
    PI = r.get("Plasticity_Index_PI(%)")
    if any(pd.isna(x) for x in [F,LL,PI]):
        return None
    GI = (max(F-35,0))*(0.2 + 0.005*max(LL-40,0)) + 0.01*max(F-15,0)*max(PI-10,0)
    return round(float(GI),2)

wide["USCS_rule"] = wide.apply(uscs_from_row, axis=1)
wide["AASHTO_GI"] = wide.apply(aashto_gi, axis=1)

# Prefer any reported class if present; else use rule
if "Soil_Classification" in wide.columns:
    wide["Soil_Class_final"] = wide["Soil_Classification"].where(
        wide["Soil_Classification"].notna() & (wide["Soil_Classification"]!=""),
        wide["USCS_rule"]
    )
else:
    wide["Soil_Class_final"] = wide["USCS_rule"]

print(wide[["SampleID","F200","Liquid_Limit_LL(%)","Plasticity_Index_PI(%)","USCS_rule","AASHTO_GI","Soil_Class_final"]].head(12))
wide.to_csv("labeled.csv", index=False)
print("✅ Saved labeled.csv with rule-based USCS & AASHTO GI.")


Parameter                   SampleID  F200  Liquid_Limit_LL(%)  \
0            Atterberg_CSU_LongBeach   NaN               48.50   
1              Atterberg_Cyprus_Intl   NaN               29.62   
2                Atterberg_Indonesia   NaN               70.00   
3            Atterberg_Namibia_GTM7b   NaN                 NaN   
4              Atterberg_UiTM_CEG454   NaN               35.80   
5             Atterberg_UiTM_ConePen   NaN               32.00   
6          Atterberg_UiTM_FullReport   NaN               48.50   
7           Atterberg_UiTM_Pahang_PL   NaN                 NaN   
8            Atterberg_UiTM_ShahAlam   NaN               26.00   
9           Atterberg_UiTM_ShahAlam2   NaN               26.00   
10                 CBR_Image_10blows   NaN                 NaN   
11                 CBR_Image_30blows   NaN                 NaN   

Parameter  Plasticity_Index_PI(%) USCS_rule AASHTO_GI  \
0                           25.60      None      None   
1                          

In [ ]:
!pip -q install xgboost lightgbm >/dev/null

import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor

# Load labeled dataset
W = pd.read_csv("labeled.csv")

# ==== CLASSIFIER ====
features_cls = [c for c in ["Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)",
                            "Cu","Cc","F200"] if c in W.columns]
cls_df = W.dropna(subset=["Soil_Class_final"])[features_cls + ["Soil_Class_final"]].copy()
for c in features_cls:
    cls_df[c] = pd.to_numeric(cls_df[c], errors="coerce")
cls_df = cls_df.dropna()

Xc = cls_df[features_cls].values
le = LabelEncoder()
yc = le.fit_transform(cls_df["Soil_Class_final"].astype(str).values)

models_cls = {
    "RF": RandomForestClassifier(n_estimators=300, random_state=42, class_weight="balanced"),
    "XGB": XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=4,
                         subsample=0.9, colsample_bytree=0.9, eval_metric="mlogloss", random_state=42),
    "LGBM": LGBMClassifier(n_estimators=500, learning_rate=0.05, subsample=0.9,
                           colsample_bytree=0.9, random_state=42)
}

print("=== Soil Classifier CV Accuracy ===")
for name, model in models_cls.items():
    if len(Xc) >= 4:
        acc = cross_val_score(model, Xc, yc, cv=min(5,len(Xc)), scoring="accuracy")
        print(f"{name}: mean={acc.mean():.3f}, scores={np.round(acc,3)}")
    else:
        print(f"{name}: not enough samples")

# ==== REGRESSOR ====
features_reg = [c for c in ["Liquid_Limit_LL(%)","Plasticity_Index_PI(%)","F200","F40","F10"] if c in W.columns]
reg_df = W.dropna(subset=["AASHTO_GI"])[features_reg + ["AASHTO_GI"]].copy()
for c in features_reg:
    reg_df[c] = pd.to_numeric(reg_df[c], errors="coerce")
reg_df = reg_df.dropna()

Xr = reg_df[features_reg].values
yr = reg_df["AASHTO_GI"].values.astype(float)

models_reg = {
    "RF": RandomForestRegressor(n_estimators=400, random_state=42),
    "XGB": XGBRegressor(n_estimators=600, learning_rate=0.05, max_depth=4,
                        subsample=0.9, colsample_bytree=0.9, random_state=42),
    "LGBM": LGBMRegressor(n_estimators=600, learning_rate=0.05, subsample=0.9,
                          colsample_bytree=0.9, random_state=42)
}

print("\n=== GI Regressor CV MAE ===")
for name, model in models_reg.items():
    if len(Xr) >= 4:
        mae = -cross_val_score(model, Xr, yr, cv=min(5,len(Xr)), scoring="neg_mean_absolute_error")
        print(f"{name}: mean={mae.mean():.3f}, scores={np.round(mae,3)}")
    else:
        print(f"{name}: not enough samples")


=== Soil Classifier CV Accuracy ===
RF: not enough samples
XGB: not enough samples
LGBM: not enough samples

=== GI Regressor CV MAE ===
RF: not enough samples
XGB: not enough samples
LGBM: not enough samples


In [ ]:
# Train Soil Classifier (Soil_Class_final) on all available data, no CV
import pandas as pd, numpy as np, joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

W = pd.read_csv("labeled.csv")

# Pick the best-available features (fallback if some are missing)
feature_priority = ["Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)",
                    "Cu","Cc","F200","F40","F10"]
features_cls = [c for c in feature_priority if c in W.columns]
print("Classifier features found:", features_cls)

# Keep rows with a label
dfc = W.dropna(subset=["Soil_Class_final"]).copy()
# Coerce numeric features and simple impute
for c in features_cls:
    dfc[c] = pd.to_numeric(dfc[c], errors="coerce")
Xc = dfc[features_cls].copy()
# require at least 2 usable features; impute remaining NaNs with column median
usable_cols = [c for c in features_cls if Xc[c].notna().sum() >= 1]
Xc = Xc[usable_cols].apply(lambda s: s.fillna(s.median()), axis=0)

y_str = dfc["Soil_Class_final"].astype(str).values
le = LabelEncoder()
yc = le.fit_transform(y_str)

print("Samples used:", len(Xc), "| Classes:", list(le.classes_))

if len(Xc) < 1:
    raise ValueError("No usable rows for classifier. Add more rows with LL/PL/PI (and %fines if possible).")

models = {
    "RF" : RandomForestClassifier(n_estimators=400, random_state=42, class_weight="balanced"),
    "XGB": XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=4,
                         subsample=0.9, colsample_bytree=0.9, eval_metric="mlogloss", random_state=42),
    "LGBM": LGBMClassifier(n_estimators=600, learning_rate=0.05, subsample=0.9,
                           colsample_bytree=0.9, random_state=42),
}

fitted = {}
for name, model in models.items():
    try:
        model.fit(Xc, yc)
        joblib.dump(model, f"{name}_soil_classifier.pkl")
        fitted[name] = True
        print(f"Saved {name}_soil_classifier.pkl")
    except Exception as e:
        fitted[name] = False
        print(f"{name} failed: {e}")

# Save label encoder + feature list actually used
joblib.dump(le, "soil_classifier_label_encoder.pkl")
pd.Series(usable_cols).to_csv("soil_classifier_features.csv", index=False)
print("Saved soil_classifier_label_encoder.pkl and soil_classifier_features.csv")


Classifier features found: ['Liquid_Limit_LL(%)', 'Plastic_Limit_PL(%)', 'Plasticity_Index_PI(%)', 'Cu', 'Cc', 'F200']
Samples used: 10 | Classes: ['CL (Low Plasticity Clay)', 'CLAY with low plasticity', 'Clay', 'Clay (High Plasticity)', 'Clay (Low Plasticity)', 'Intermediate Clay', 'Non-plastic / Silt', 'Plastic Clay', 'SP']
Saved RF_soil_classifier.pkl
Saved XGB_soil_classifier.pkl
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 10, number of used features: 0
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -1.609438
[Ligh

In [ ]:
# Train GI regressor on all available data, no CV
import pandas as pd, numpy as np, joblib
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

W = pd.read_csv("labeled.csv")

# Features (use what exists)
feature_priority_reg = ["Liquid_Limit_LL(%)","Plasticity_Index_PI(%)","F200","F40","F10"]
features_reg = [c for c in feature_priority_reg if c in W.columns]
print("Regressor features found:", features_reg)

dfr = W.dropna(subset=["AASHTO_GI"]).copy()
for c in features_reg:
    dfr[c] = pd.to_numeric(dfr[c], errors="coerce")
Xr = dfr[features_reg].copy()
# Impute NaNs with column median
usable_cols_r = [c for c in features_reg if Xr[c].notna().sum() >= 1]
Xr = Xr[usable_cols_r].apply(lambda s: s.fillna(s.median()), axis=0)
yr = pd.to_numeric(dfr["AASHTO_GI"], errors="coerce")
mask = yr.notna()
Xr, yr = Xr[mask], yr[mask]

print("Samples used:", len(Xr))

if len(Xr) < 1:
    raise ValueError("No usable rows for GI regressor. Add rows with LL/PI and F200 (or %Fines).")

models_r = {
    "RF" : RandomForestRegressor(n_estimators=500, random_state=42),
    "XGB": XGBRegressor(n_estimators=700, learning_rate=0.05, max_depth=4,
                        subsample=0.9, colsample_bytree=0.9, random_state=42),
    "LGBM": LGBMRegressor(n_estimators=700, learning_rate=0.05, subsample=0.9,
                          colsample_bytree=0.9, random_state=42),
}

for name, model in models_r.items():
    try:
        model.fit(Xr, yr)
        joblib.dump(model, f"{name}_gi_regressor.pkl")
        print(f"Saved {name}_gi_regressor.pkl")
    except Exception as e:
        print(f"{name} failed: {e}")

pd.Series(usable_cols_r).to_csv("gi_regressor_features.csv", index=False)
print("Saved gi_regressor_features.csv")


Regressor features found: ['Liquid_Limit_LL(%)', 'Plasticity_Index_PI(%)', 'F200']
Samples used: 0


ValueError: No usable rows for GI regressor. Add rows with LL/PI and F200 (or %Fines).

In [ ]:
import pandas as pd
W = pd.read_csv("labeled.csv")
cols = ["F200","Liquid_Limit_LL(%)","Plasticity_Index_PI(%)","AASHTO_GI"]
print(W[cols].isna().sum())
print("Rows with ALL of F200, LL, PI:", ((~W["F200"].isna()) & (~W["Liquid_Limit_LL(%)"].isna()) & (~W["Plasticity_Index_PI(%)"].isna())).sum())
print("Non-null AASHTO_GI:", W["AASHTO_GI"].notna().sum())


F200                      39
Liquid_Limit_LL(%)        32
Plasticity_Index_PI(%)    33
AASHTO_GI                 40
dtype: int64
Rows with ALL of F200, LL, PI: 0
Non-null AASHTO_GI: 0


In [ ]:
import re, numpy as np, pandas as pd

W = pd.read_csv("labeled.csv")

# If F200 is missing or mostly NaN, try to derive it again:
needs_f200 = ("F200" not in W.columns) or (W["F200"].isna().mean() > 0.5)

if needs_f200:
    # Look for any column that seems to be "% passing 0.075 mm" or "fines"
    candidates = [c for c in W.columns if isinstance(c, str)]
    mm075_cols = [c for c in candidates if ("0.075" in c.replace(" ", "")) and ("%Pass" in c or "Percent_Passing" in c or "%Passing" in c or "% Finer" in c or "Finer" in c)]
    # Fallbacks people often use:
    common_fines = [c for c in candidates if c.strip().lower() in {"%fines","fines_%","fines (%)","percent fines","fines"}]
    pick = None
    if "F200" in W.columns: pick = "F200"
    elif mm075_cols:       pick = mm075_cols[0]
    elif common_fines:     pick = common_fines[0]
    if pick:
        W["F200"] = pd.to_numeric(W[pick], errors="coerce")
        print("Derived F200 from:", pick)
    else:
        print("Could not auto-derive F200 — please ensure a %passing(0.075 mm) or %Fines column exists.")

# Recompute AASHTO_GI wherever F200, LL, PI exist
def gi_row(r):
    F  = r.get("F200"); LL = r.get("Liquid_Limit_LL(%)"); PI = r.get("Plasticity_Index_PI(%)")
    if any(pd.isna(x) for x in [F,LL,PI]): return np.nan
    return (max(F-35,0))*(0.2 + 0.005*max(LL-40,0)) + 0.01*max(F-15,0)*max(PI-10,0)

W["AASHTO_GI"] = W.apply(gi_row, axis=1).astype(float).round(2)

# Save back
W.to_csv("labeled.csv", index=False)
print("Re-saved labeled.csv — non-null GI rows:", W["AASHTO_GI"].notna().sum())


Derived F200 from: F200
Re-saved labeled.csv — non-null GI rows: 0


In [ ]:
import pandas as pd
cbr_df = pd.read_csv("/content/cbr_data.csv")
print(cbr_df)


FileNotFoundError: [Errno 2] No such file or directory: '/content/cbr_data.csv'

In [ ]:
import pandas as pd

cbr_df = pd.read_csv("CBR_Data__from_reports_.csv")
print(cbr_df)


           SampleID  MDD_gcc  OMC_pct  Blows  CBR_2p5_pct  Adopted_CBR_pct
0  CBR_demo_10blows    2.204      8.0     10         11.5             11.5
1  CBR_demo_30blows    2.204      8.0     30         52.5             52.5
2  CBR_demo_65blows    2.204      8.0     65         68.0             68.0


In [ ]:
# Train 3 regressors on your CBR data and save them
!pip -q install xgboost lightgbm >/dev/null

import pandas as pd, numpy as np, joblib
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# use the cbr_df you already loaded
df = cbr_df.copy()

features = ["MDD_gcc","OMC_pct","Blows"]
if "CBR_2p5_pct" in df.columns:
    features.append("CBR_2p5_pct")

X = df[features].copy().astype(float).fillna(df[features].median(numeric_only=True))
y = pd.to_numeric(df["Adopted_CBR_pct"], errors="coerce")
mask = y.notna()
X, y = X[mask], y[mask]

print("Samples used:", len(X))
print("Features:", features)

models = {
    "RF" : RandomForestRegressor(n_estimators=400, random_state=42),
    "XGB": XGBRegressor(n_estimators=600, learning_rate=0.05, max_depth=3,
                        subsample=0.9, colsample_bytree=0.9, random_state=42),
    "LGBM": LGBMRegressor(n_estimators=600, learning_rate=0.05, subsample=0.9,
                          colsample_bytree=0.9, random_state=42),
}

for name, model in models.items():
    model.fit(X, y)
    pred = model.predict(X)
    print(f"{name}: MAE(train)={mean_absolute_error(y, pred):.3f} | R2(train)={r2_score(y, pred):.3f}")
    joblib.dump(model, f"{name}_cbr_regressor.pkl")
    print(f"Saved {name}_cbr_regressor.pkl")

pd.Series(features).to_csv("cbr_regressor_features.csv", index=False)
print("Saved cbr_regressor_features.csv")


Samples used: 3
Features: ['MDD_gcc', 'OMC_pct', 'Blows', 'CBR_2p5_pct']
RF: MAE(train)=7.730 | R2(train)=0.874
Saved RF_cbr_regressor.pkl
XGB: MAE(train)=0.001 | R2(train)=1.000
Saved XGB_cbr_regressor.pkl
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 3, number of used features: 0
[LightGBM] [Info] Start training from score 44.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split

In [ ]:
import joblib, numpy as np, pandas as pd

feat = pd.read_csv("cbr_regressor_features.csv", header=None)[0].tolist()
model = joblib.load("RF_cbr_regressor.pkl")  # try XGB/LGBM as well

# take the first row from your dataset
row = cbr_df.iloc[0][feat].astype(float).to_frame().T.fillna(cbr_df[feat].median(numeric_only=True))
pred = float(model.predict(row)[0])
print("Input:", row.to_dict(orient="records")[0])
print("Predicted Adopted CBR (%):", round(pred, 2))


KeyError: "['0'] not in index"

In [ ]:
import joblib, pandas as pd, numpy as np

# Load model + saved feature list
model = joblib.load("RF_cbr_regressor.pkl")   # you can switch to XGB/LGBM if you want
feat_saved = pd.read_csv("cbr_regressor_features.csv", header=None)[0].astype(str).str.strip().tolist()

# Make sure our dataframe columns are trimmed
cbr_df = cbr_df.copy()
cbr_df.columns = [c.strip() for c in cbr_df.columns]

print("Model expects features:", feat_saved)
print("CSV has columns:", list(cbr_df.columns))

# Split into present/missing features
present = [f for f in feat_saved if f in cbr_df.columns]
missing = [f for f in feat_saved if f not in cbr_df.columns]
print("Present:", present)
print("Missing:", missing)

# Build one input row
row = cbr_df.iloc[[0]][present].astype(float)

# For any missing expected feature, add a column filled with the median of that feature (or 0 if N/A)
for f in missing:
    # if the feature isn't in the dataset at all (e.g., optional CBR_2p5_pct), use the median of any present col
    row[f] = float(cbr_df[present].astype(float).median(numeric_only=True)) if len(present)>0 else 0.0

# Reorder columns exactly as the model expects
row = row[feat_saved].fillna(row.median(numeric_only=True))

# Predict
pred = float(model.predict(row.values)[0])
print("Input used:", row.to_dict(orient="records")[0])
print("Predicted Adopted CBR (%):", round(pred, 2))


Model expects features: ['0', 'MDD_gcc', 'OMC_pct', 'Blows', 'CBR_2p5_pct']
CSV has columns: ['SampleID', 'MDD_gcc', 'OMC_pct', 'Blows', 'CBR_2p5_pct', 'Adopted_CBR_pct']
Present: ['MDD_gcc', 'OMC_pct', 'Blows', 'CBR_2p5_pct']
Missing: ['0']


TypeError: cannot convert the series to <class 'float'>

In [ ]:
import pandas as pd

# This should be the same list you used for training
features = ["MDD_gcc","OMC_pct","Blows","CBR_2p5_pct"]  # remove CBR_2p5_pct if you didn't train with it
pd.Series(features).to_csv("cbr_regressor_features.csv", index=False, header=False)

# Verify file contents
print(pd.read_csv("cbr_regressor_features.csv", header=None)[0].tolist())


['MDD_gcc', 'OMC_pct', 'Blows', 'CBR_2p5_pct']


In [ ]:
import joblib, pandas as pd, numpy as np

# Load model + corrected feature list
model = joblib.load("RF_cbr_regressor.pkl")  # try XGB/LGBM too if you want
feat_saved = pd.read_csv("cbr_regressor_features.csv", header=None)[0].astype(str).str.strip().tolist()

# Clean column names
cbr_df = cbr_df.copy()
cbr_df.columns = [c.strip() for c in cbr_df.columns]

print("Model expects features:", feat_saved)
print("CSV has columns:", list(cbr_df.columns))

# Split present/missing
present = [f for f in feat_saved if f in cbr_df.columns]
missing = [f for f in feat_saved if f not in cbr_df.columns]
print("Present:", present, " Missing:", missing)

# Build one input row from first sample
row = cbr_df.iloc[[0]][present].astype(float)

# For any missing expected feature (should be none now), fill with overall median of present columns
if missing:
    fallback_val = float(cbr_df[present].astype(float).median(numeric_only=True))
    for f in missing:
        row[f] = fallback_val

# Reorder exactly as model expects and fill NaNs
row = row[feat_saved].fillna(row.median(numeric_only=True))

pred = float(model.predict(row.values)[0])
print("Input used:", row.to_dict(orient="records")[0])
print("Predicted Adopted CBR (%):", round(pred, 2))


Model expects features: ['MDD_gcc', 'OMC_pct', 'Blows', 'CBR_2p5_pct']
CSV has columns: ['SampleID', 'MDD_gcc', 'OMC_pct', 'Blows', 'CBR_2p5_pct', 'Adopted_CBR_pct']
Present: ['MDD_gcc', 'OMC_pct', 'Blows', 'CBR_2p5_pct']  Missing: []
Input used: {'MDD_gcc': 2.204, 'OMC_pct': 8.0, 'Blows': 10.0, 'CBR_2p5_pct': 11.5}
Predicted Adopted CBR (%): 24.0


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [ ]:
import joblib, pandas as pd

# Load model + features
feat = pd.read_csv("cbr_regressor_features.csv", header=None)[0].astype(str).str.strip().tolist()
rf  = joblib.load("RF_cbr_regressor.pkl")   # try XGB/LGBM too if you saved them

# Ensure columns/ordering
row = cbr_df.iloc[[0]][feat].astype(float).fillna(cbr_df[feat].median(numeric_only=True))

# Predict using a DataFrame (no warning, uses feature names)
pred = float(rf.predict(row)[0])
print("Input used:", row.to_dict(orient="records")[0])
print("Predicted Adopted CBR (%):", round(pred, 2))


Input used: {'MDD_gcc': 2.204, 'OMC_pct': 8.0, 'Blows': 10.0, 'CBR_2p5_pct': 11.5}
Predicted Adopted CBR (%): 24.0


In [ ]:
import numpy as np, pandas as pd, joblib
from sklearn.metrics import mean_absolute_error, r2_score

feat = pd.read_csv("cbr_regressor_features.csv", header=None)[0].astype(str).str.strip().tolist()
dfX  = cbr_df[feat].astype(float).fillna(cbr_df[feat].median(numeric_only=True))
y    = cbr_df["Adopted_CBR_pct"].astype(float).values

models = {}
for name in ["RF","XGB","LGBM"]:
    try:
        models[name] = joblib.load(f"{name}_cbr_regressor.pkl")
    except:
        pass

for name, mdl in models.items():
    yhat = mdl.predict(dfX)
    print(f"{name}: MAE={mean_absolute_error(y,yhat):.2f} | R2={r2_score(y,yhat):.2f} | preds={np.round(yhat,2)} | true={y}")


RF: MAE=7.73 | R2=0.87 | preds=[24.   48.07 61.74] | true=[11.5 52.5 68. ]
XGB: MAE=0.00 | R2=1.00 | preds=[11.5 52.5 68. ] | true=[11.5 52.5 68. ]
LGBM: MAE=21.67 | R2=0.00 | preds=[44. 44. 44.] | true=[11.5 52.5 68. ]


In [ ]:
# ====== Gradio Frontend with Charts (Soil Class + CBR) ======
!pip -q install gradio lightgbm xgboost plotly >/dev/null

import math, joblib, json
import numpy as np
import pandas as pd
import gradio as gr
from pathlib import Path
import plotly.graph_objects as go

# ---------- Utility ----------
def try_load(path):
    p = Path(path)
    if p.exists():
        try:
            return joblib.load(p)
        except Exception as e:
            print(f"Failed to load {path}: {e}")
    return None

# ---------- Load Soil Classifier ----------
soil_models = {"RF": try_load("RF_soil_classifier.pkl"),
               "XGB": try_load("XGB_soil_classifier.pkl"),
               "LGBM": try_load("LGBM_soil_classifier.pkl")}
soil_le = try_load("soil_classifier_label_encoder.pkl")
soil_features = pd.read_csv("soil_classifier_features.csv", header=None)[0].tolist() \
                 if Path("soil_classifier_features.csv").exists() else []
soil_model = next((m for m in soil_models.values() if m is not None), None)
soil_model_name = next((n for n,m in soil_models.items() if m is not None), "None")

# ---------- Load CBR Regressor ----------
cbr_models = {"RF": try_load("RF_cbr_regressor.pkl"),
              "XGB": try_load("XGB_cbr_regressor.pkl"),
              "LGBM": try_load("LGBM_cbr_regressor.pkl")}
cbr_features = pd.read_csv("cbr_regressor_features.csv", header=None)[0].tolist() \
               if Path("cbr_regressor_features.csv").exists() else []
cbr_model = next((m for m in cbr_models.values() if m is not None), None)
cbr_model_name = next((n for n,m in cbr_models.items() if m is not None), "None")

# ---------- Rules / helpers ----------
def a_line(LL): return 0.73*(LL-20)

def uscs_rule(LL, PI, F200=None, Cu=None, Cc=None):
    try:
        F  = float(F200) if F200 not in [None,""] else None
        LL = float(LL)   if LL   not in [None,""] else None
        PI = float(PI)   if PI   not in [None,""] else None
        Cu = float(Cu)   if Cu   not in [None,""] else None
        Cc = float(Cc)   if Cc   not in [None,""] else None
    except: return "Unknown"

    if F is None and (LL is not None and PI is not None): F = 51
    if F is None: return "Unknown"

    if F < 50:
        if F < 5:
            if (Cu is not None and Cc is not None) and (Cu>6 and 1<Cc<3): return "SW"
            return "SP"
        elif F > 12:
            if LL is None or PI is None: return "S(with fines)"
            return "SC" if PI >= a_line(LL) else "SM"
        else:
            if (Cu is not None and Cc is not None) and (Cu>6 and 1<Cc<3): return "SW-SM"
            return "SP-SM"
    else:
        if LL is None or PI is None: return "Fine (unknown)"
        return ("CL" if PI >= a_line(LL) else "ML") if LL<50 else ("CH" if PI >= a_line(LL) else "MH")

# Plasticity chart
def plasticity_chart(LL, PI):
    try:
        LL = float(LL) if LL not in [None,""] else None
        PI = float(PI) if PI not in [None,""] else None
    except:
        LL = PI = None

    ll_axis = np.linspace(0, 100, 201)
    a = 0.73*(ll_axis-20)
    u = 0.9*(ll_axis-8)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ll_axis, y=a, mode="lines", name="A-line", line=dict(color="#2563eb")))
    fig.add_trace(go.Scatter(x=ll_axis, y=u, mode="lines", name="U-line", line=dict(color="#94a3b8", dash="dash")))
    fig.update_xaxes(title="Liquid Limit, LL (%)", range=[0,100], gridcolor="#e5e7eb")
    fig.update_yaxes(title="Plasticity Index, PI (%)", range=[0,70], gridcolor="#e5e7eb")
    fig.update_layout(height=420, template="plotly_white", legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

    if LL is not None and PI is not None:
        fig.add_trace(go.Scatter(x=[LL], y=[PI], mode="markers+text",
                                 text=["Sample"], textposition="top center",
                                 marker=dict(size=10, color="#10b981"), name="Your point"))
    return fig

# CBR curve chart
def cbr_curve_chart(mdd, omc, blows, cbr25, model, feat):
    xs = list(range(5, 70, 5))  # blows range for curve
    preds = []
    for b in xs:
        row = {"MDD_gcc":mdd, "OMC_pct":omc, "Blows":b, "CBR_2p5_pct":cbr25}
        X = pd.DataFrame([row])
        for f in feat:
            if f not in X.columns: X[f] = np.nan
        X = X[feat].apply(pd.to_numeric, errors="coerce").fillna(X.median(numeric_only=True))
        if model is not None:
            preds.append(float(model.predict(X)[0]))
        else:
            # small heuristic if no model
            base = (float(mdd)-1.6)*80 if mdd not in [None,""] else 30
            comp = (10-abs(float(omc)-8))*1.2 if omc not in [None,""] else 8
            effort = (b/65)*40
            extra = float(cbr25) if cbr25 not in [None,""] else 0
            preds.append(max(2.0, base + comp + effort*0.8 + 0.4*extra))

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=xs, y=preds, mode="lines+markers",
                             name="Predicted CBR vs Blows", line=dict(color="#7c3aed")))
    # Highlight selection
    if blows not in [None,""]:
        # predict at selected blows for marker
        row_sel = {"MDD_gcc":mdd, "OMC_pct":omc, "Blows":blows, "CBR_2p5_pct":cbr25}
        Xs = pd.DataFrame([row_sel])
        for f in feat:
            if f not in Xs.columns: Xs[f] = np.nan
        Xs = Xs[feat].apply(pd.to_numeric, errors="coerce").fillna(Xs.median(numeric_only=True))
        y_sel = float(model.predict(Xs)[0]) if model is not None else None
        if y_sel is not None:
            fig.add_trace(go.Scatter(x=[blows], y=[y_sel], mode="markers",
                                     marker=dict(size=12, color="#ef4444"),
                                     name="Selected"))
    fig.update_xaxes(title="Compaction Blows (#)", gridcolor="#e5e7eb")
    fig.update_yaxes(title="Predicted CBR (%)", gridcolor="#e5e7eb")
    fig.update_layout(height=420, template="plotly_white")
    return fig

# ---------- Predictors ----------
def predict_soil_and_plot(LL, PL, PI, F200, Cu, Cc):
    if (PI is None or PI=="") and all(x not in [None,""] for x in [LL, PL]):
        try: PI = float(LL)-float(PL)
        except: PI = None

    if soil_model is not None and soil_le is not None and soil_features:
        row = {}
        for f in soil_features:
            fl = f.lower()
            if fl.startswith("liquid"): row[f] = LL
            elif fl.startswith("plastic_limit"): row[f] = PL
            elif "plasticity_index" in fl: row[f] = PI
            elif fl == "f200": row[f] = F200
            elif fl == "cu":   row[f] = Cu
            elif fl == "cc":   row[f] = Cc
            else: row[f] = None
        X = pd.DataFrame([row])[soil_features].apply(pd.to_numeric, errors="coerce")
        X = X.fillna(X.median(numeric_only=True))
        try:
            yhat = soil_model.predict(X)[0]
            label = soil_le.inverse_transform([yhat])[0]
            note = f"Model: {soil_model_name} | Features: {', '.join(soil_features)}"
        except Exception as e:
            label = uscs_rule(LL, PI, F200, Cu, Cc)
            note = f"Model error ({e}) → rule-based fallback."
    else:
        label = uscs_rule(LL, PI, F200, Cu, Cc)
        note  = "Model not found → rule-based USCS applied."

    fig = plasticity_chart(LL, PI)
    return label, f"**Info:** {note}", fig

def predict_cbr_and_plot(MDD, OMC, Blows, CBR25):
    row = {"MDD_gcc":MDD, "OMC_pct":OMC, "Blows":Blows, "CBR_2p5_pct":CBR25}
    if cbr_model is not None and cbr_features:
        X = pd.DataFrame([row])
        for f in cbr_features:
            if f not in X.columns: X[f] = np.nan
        X = X[cbr_features].apply(pd.to_numeric, errors="coerce").fillna(X.median(numeric_only=True))
        try:
            yhat = float(cbr_model.predict(X)[0])
            note = f"Model: {cbr_model_name} | Features: {', '.join(cbr_features)}"
        except Exception as e:
            yhat = None; note = f"Model error ({e}) → heuristic used."
    else:
        yhat = None; note = "Model not found → heuristic used."

    if yhat is None:
        base = (float(MDD)-1.6)*80 if MDD not in [None,""] else 30
        comp = (10-abs(float(OMC)-8))*1.2 if OMC not in [None,""] else 8
        effort = (float(Blows)/65)*40 if Blows not in [None,""] else 20
        extra = float(CBR25) if CBR25 not in [None,""] else 0
        yhat = max(2.0, base + comp + effort*0.8 + 0.4*extra)

    fig = cbr_curve_chart(MDD, OMC, Blows, CBR25, cbr_model, cbr_features if cbr_features else ["MDD_gcc","OMC_pct","Blows","CBR_2p5_pct"])
    return round(float(yhat),2), f"**Info:** {note}", fig

# ---------- UI ----------
theme = gr.themes.Soft(primary_hue="indigo", neutral_hue="slate")

with gr.Blocks(theme=theme, fill_height=True, title="Soil AI — Classifier & CBR (with Charts)") as demo:
    gr.Markdown("<div style='text-align:center'><h1>Soil AI — Classifier & CBR</h1><p style='color:#475569'>Interactive predictions with visual charts</p></div>")

    with gr.Tab("Soil Classifier"):
        with gr.Row():
            with gr.Column():
                LL = gr.Number(label="Liquid Limit (LL, %)", value=40)
                PL = gr.Number(label="Plastic Limit (PL, %) — optional", value=25)
                PI = gr.Number(label="Plasticity Index (PI, %) — leave blank to auto LL-PL", value=None)
            with gr.Column():
                F200 = gr.Number(label="% Passing No.200 (0.075 mm) — optional", value=None)
                Cu = gr.Number(label="Cu — optional", value=None)
                Cc = gr.Number(label="Cc — optional", value=None)
        btn1 = gr.Button("Predict Soil Class", variant="primary")
        out_class = gr.Textbox(label="Predicted USCS / Soil Class", interactive=False)
        out_note  = gr.Markdown()
        soil_fig  = gr.Plot(label="Plasticity Chart")
        btn1.click(predict_soil_and_plot, inputs=[LL, PL, PI, F200, Cu, Cc],
                   outputs=[out_class, out_note, soil_fig])

    with gr.Tab("CBR Predictor"):
        with gr.Row():
            with gr.Column():
                MDD = gr.Number(label="MDD (g/cc)", value=2.204)
                OMC = gr.Number(label="OMC (%)", value=8.0)
            with gr.Column():
                Blows = gr.Number(label="Compaction Blows (#) — 10/30/65", value=30)
                CBR25 = gr.Number(label="CBR at 2.5 mm (%) — optional", value=None)
        btn2 = gr.Button("Predict CBR (%)", variant="primary")
        out_cbr = gr.Number(label="Predicted Adopted CBR (%)", interactive=False, precision=2)
        out_cbr_note = gr.Markdown()
        cbr_fig = gr.Plot(label="CBR vs Blows (predicted)")
        btn2.click(predict_cbr_and_plot, inputs=[MDD, OMC, Blows, CBR25],
                   outputs=[out_cbr, out_cbr_note, cbr_fig])

    with gr.Accordion("Status", open=False):
        gr.Markdown(
            f"- Soil model: **{soil_model_name}** | Features: `{', '.join(soil_features) if soil_features else '—'}`  \n"
            f"- CBR model: **{cbr_model_name}** | Features: `{', '.join(cbr_features) if cbr_features else '—'}`  \n"
            "<small>Tip: add more rows and retrain to improve curves and accuracy.</small>"
        )

demo.queue().launch(share=False)


ERROR: Operation cancelled by user
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
# ====== Soil AI Frontend (Classifier + CBR + Gradation Charts) ======
!pip -q install gradio lightgbm xgboost plotly >/dev/null

import math, joblib
import numpy as np
import pandas as pd
import gradio as gr
from pathlib import Path
import plotly.graph_objects as go

# --------- helpers ----------
def try_load(path):
    p = Path(path)
    if p.exists():
        try: return joblib.load(p)
        except Exception as e: print(f"Load fail {path}: {e}")
    return None

# --------- load trained assets (soil + cbr) ----------
soil_models = {"RF": try_load("RF_soil_classifier.pkl"),
               "XGB": try_load("XGB_soil_classifier.pkl"),
               "LGBM": try_load("LGBM_soil_classifier.pkl")}
soil_le = try_load("soil_classifier_label_encoder.pkl")
soil_features = pd.read_csv("soil_classifier_features.csv", header=None)[0].tolist() \
                 if Path("soil_classifier_features.csv").exists() else []
soil_model = next((m for m in soil_models.values() if m is not None), None)
soil_model_name = next((n for n,m in soil_models.items() if m is not None), "None")

cbr_models = {"RF": try_load("RF_cbr_regressor.pkl"),
              "XGB": try_load("XGB_cbr_regressor.pkl"),
              "LGBM": try_load("LGBM_cbr_regressor.pkl")}
cbr_features = pd.read_csv("cbr_regressor_features.csv", header=None)[0].tolist() \
               if Path("cbr_regressor_features.csv").exists() else []
cbr_model = next((m for m in cbr_models.values() if m is not None), None)
cbr_model_name = next((n for n,m in cbr_models.items() if m is not None), "None")

# --------- rules / small utils ----------
def a_line(LL): return 0.73*(LL-20)
def uscs_rule(LL, PI, F200=None, Cu=None, Cc=None):
    try:
        F  = float(F200) if F200 not in [None,""] else None
        LL = float(LL)   if LL   not in [None,""] else None
        PI = float(PI)   if PI   not in [None,""] else None
        Cu = float(Cu)   if Cu   not in [None,""] else None
        Cc = float(Cc)   if Cc   not in [None,""] else None
    except: return "Unknown"
    if F is None and (LL is not None and PI is not None): F = 51
    if F is None: return "Unknown"
    if F < 50:
        if F < 5:
            if (Cu is not None and Cc is not None) and (Cu>6 and 1<Cc<3): return "SW"
            return "SP"
        elif F > 12:
            if LL is None or PI is None: return "S(with fines)"
            return "SC" if PI >= a_line(LL) else "SM"
        else:
            if (Cu is not None and Cc is not None) and (Cu>6 and 1<Cc<3): return "SW-SM"
            return "SP-SM"
    else:
        if LL is None or PI is None: return "Fine (unknown)"
        return ("CL" if PI >= a_line(LL) else "ML") if LL<50 else ("CH" if PI >= a_line(LL) else "MH")

def plasticity_chart(LL, PI):
    try: LL = float(LL) if LL not in [None,""] else None; PI = float(PI) if PI not in [None,""] else None
    except: LL = PI = None
    ll = np.linspace(0,100,201); a = 0.73*(ll-20); u = 0.9*(ll-8)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ll, y=a, mode="lines", name="A-line", line=dict(color="#2563eb")))
    fig.add_trace(go.Scatter(x=ll, y=u, mode="lines", name="U-line", line=dict(color="#94a3b8", dash="dash")))
    if LL is not None and PI is not None:
        fig.add_trace(go.Scatter(x=[LL], y=[PI], mode="markers+text", text=["Sample"],
                                 textposition="top center", marker=dict(size=10, color="#10b981")))
    fig.update_xaxes(title="Liquid Limit, LL (%)", range=[0,100], type="linear", gridcolor="#e5e7eb")
    fig.update_yaxes(title="Plasticity Index, PI (%)", range=[0,70],  gridcolor="#e5e7eb")
    fig.update_layout(height=420, template="plotly_white", legend=dict(orientation="h", y=1.02, x=1))
    return fig

def cbr_curve_chart(mdd, omc, blows, cbr25, model, feat):
    xs = list(range(5, 70, 5)); preds = []
    for b in xs:
        row = {"MDD_gcc":mdd, "OMC_pct":omc, "Blows":b, "CBR_2p5_pct":cbr25}
        X = pd.DataFrame([row])
        for f in feat:
            if f not in X.columns: X[f] = np.nan
        X = X[feat].apply(pd.to_numeric, errors="coerce").fillna(X.median(numeric_only=True))
        if model is not None: preds.append(float(model.predict(X)[0]))
        else:
            base = (float(mdd)-1.6)*80 if mdd not in [None,""] else 30
            comp = (10-abs(float(omc)-8))*1.2 if omc not in [None,""] else 8
            effort = (b/65)*40; extra = float(cbr25) if cbr25 not in [None,""] else 0
            preds.append(max(2.0, base + comp + effort*0.8 + 0.4*extra))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=xs, y=preds, mode="lines+markers", name="Predicted CBR vs Blows", line=dict(color="#7c3aed")))
    if blows not in [None,""]:
        row_sel = {"MDD_gcc":mdd, "OMC_pct":omc, "Blows":blows, "CBR_2p5_pct":cbr25}
        Xs = pd.DataFrame([row_sel])
        for f in feat:
            if f not in Xs.columns: Xs[f] = np.nan
        Xs = Xs[feat].apply(pd.to_numeric, errors="coerce").fillna(Xs.median(numeric_only=True))
        y_sel = float(model.predict(Xs)[0]) if model is not None else None
        if y_sel is not None:
            fig.add_trace(go.Scatter(x=[blows], y=[y_sel], mode="markers", marker=dict(size=12, color="#ef4444"), name="Selected"))
    fig.update_xaxes(title="Compaction Blows (#)", gridcolor="#e5e7eb")
    fig.update_yaxes(title="Predicted CBR (%)",   gridcolor="#e5e7eb")
    fig.update_layout(height=420, template="plotly_white"); return fig

# --------- gradation math (log-scale interpolation) ----------
def interpolate_dx(sizes_mm, passing_pct, target_pct):
    # sizes in mm; passing in %; returns D_target using log-size interpolation
    x = np.array(sizes_mm, dtype=float)
    y = np.array(passing_pct, dtype=float)
    # sort by size descending -> ascending for interp
    order = np.argsort(x)
    x, y = x[order], y[order]
    # require target within range
    if not (y.min() <= target_pct <= y.max()): return None
    # log10(size) vs % passing linear interpolation
    logx = np.log10(x)
    return float(10 ** np.interp(target_pct, y, logx))

def gradation_plot(df):
    # df columns: Sieve_mm, Percent_Passing
    g = df.dropna().copy()
    g["Sieve_mm"] = pd.to_numeric(g["Sieve_mm"], errors="coerce")
    g["Percent_Passing"] = pd.to_numeric(g["Percent_Passing"], errors="coerce")
    g = g.dropna().sort_values("Sieve_mm", ascending=False)
    if g.empty:
        return None, "No valid points.", None, None, None, None, None

    # compute D-values
    D10 = interpolate_dx(g["Sieve_mm"], g["Percent_Passing"], 10)
    D30 = interpolate_dx(g["Sieve_mm"], g["Percent_Passing"], 30)
    D60 = interpolate_dx(g["Sieve_mm"], g["Percent_Passing"], 60)
    Cu = (D60 / D10) if (D60 and D10 and D10>0) else None
    Cc = ((D30**2) / (D10*D60)) if (D30 and D10 and D60 and D10>0 and D60>0) else None

    # plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=g["Sieve_mm"], y=g["Percent_Passing"], mode="lines+markers",
                             name="% Passing", line=dict(color="#0ea5e9")))
    # Vertical lines for D-values
    for name, val in [("D10",D10),("D30",D30),("D60",D60)]:
        if val:
            fig.add_trace(go.Scatter(x=[val,val], y=[0,100], mode="lines",
                                     line=dict(dash="dot", color="#94a3b8"), name=name))
    fig.update_xaxes(title="Particle Size (mm)", type="log", autorange="reversed", gridcolor="#e5e7eb")
    fig.update_yaxes(title="% Passing", range=[0,100], gridcolor="#e5e7eb")
    fig.update_layout(height=460, template="plotly_white", legend=dict(orientation="h", y=1.02, x=1))

    return fig, "", D10, D30, D60, Cu, Cc

# --------- predictors for UI ----------
def predict_soil_and_plot(LL, PL, PI, F200, Cu, Cc):
    if (PI is None or PI=="") and all(x not in [None,""] for x in [LL, PL]):
        try: PI = float(LL)-float(PL)
        except: PI = None
    note = ""
    if soil_model is not None and soil_le is not None and soil_features:
        row = {}
        for f in soil_features:
            fl = f.lower()
            if fl.startswith("liquid"): row[f] = LL
            elif fl.startswith("plastic_limit"): row[f] = PL
            elif "plasticity_index" in fl: row[f] = PI
            elif fl=="f200": row[f] = F200
            elif fl=="cu":   row[f] = Cu
            elif fl=="cc":   row[f] = Cc
            else: row[f] = None
        X = pd.DataFrame([row])[soil_features].apply(pd.to_numeric, errors="coerce").fillna(method="pad").fillna(0)
        try:
            yhat = soil_model.predict(X.fillna(X.median(numeric_only=True)))[0]
            label = soil_le.inverse_transform([yhat])[0]
            note = f"Model: {soil_model_name} | Features: {', '.join(soil_features)}"
        except Exception as e:
            label = uscs_rule(LL, PI, F200, Cu, Cc)
            note  = f"Model error ({e}) → rule-based fallback."
    else:
        label = uscs_rule(LL, PI, F200, Cu, Cc)
        note  = "Model not found → rule-based USCS applied."
    return label, f"**Info:** {note}", plasticity_chart(LL, PI)

def predict_cbr_and_plot(MDD, OMC, Blows, CBR25):
    row = {"MDD_gcc":MDD, "OMC_pct":OMC, "Blows":Blows, "CBR_2p5_pct":CBR25}
    note = ""
    if cbr_model is not None and cbr_features:
        X = pd.DataFrame([row])
        for f in cbr_features:
            if f not in X.columns: X[f] = np.nan
        X = X[cbr_features].apply(pd.to_numeric, errors="coerce").fillna(X.median(numeric_only=True))
        try:
            yhat = float(cbr_model.predict(X)[0]); note = f"Model: {cbr_model_name} | Features: {', '.join(cbr_features)}"
        except Exception as e:
            yhat = None; note = f"Model error ({e}) → heuristic used."
    else:
        yhat = None; note = "Model not found → heuristic used."
    if yhat is None:
        base = (float(MDD)-1.6)*80 if MDD not in [None,""] else 30
        comp = (10-abs(float(OMC)-8))*1.2 if OMC not in [None,""] else 8
        effort = (float(Blows)/65)*40 if Blows not in [None,""] else 20
        extra = float(CBR25) if CBR25 not in [None,""] else 0
        yhat = max(2.0, base + comp + effort*0.8 + 0.4*extra)
    fig = cbr_curve_chart(MDD, OMC, Blows, CBR25, cbr_model, cbr_features if cbr_features else ["MDD_gcc","OMC_pct","Blows","CBR_2p5_pct"])
    return round(float(yhat),2), f"**Info:** {note}", fig

# --------- UI (three tabs) ----------
theme = gr.themes.Soft(primary_hue="indigo", neutral_hue="slate")

with gr.Blocks(theme=theme, fill_height=True, title="Soil AI — Classifier, CBR & Gradation") as demo:
    gr.Markdown("<div style='text-align:center'><h1>Soil AI — Classifier, CBR & Gradation</h1><p style='color:#475569'>Interactive predictions with charts</p></div>")

    with gr.Tab("Soil Classifier"):
        with gr.Row():
            with gr.Column():
                LL = gr.Number(label="Liquid Limit (LL, %)", value=40)
                PL = gr.Number(label="Plastic Limit (PL, %) — optional", value=25)
                PI = gr.Number(label="Plasticity Index (PI, %) — leave blank to auto LL-PL", value=None)
            with gr.Column():
                F200 = gr.Number(label="% Passing No.200 (0.075 mm) — optional", value=None)
                Cu = gr.Number(label="Cu — optional", value=None)
                Cc = gr.Number(label="Cc — optional", value=None)
        btn1 = gr.Button("Predict Soil Class", variant="primary")
        out_class = gr.Textbox(label="Predicted USCS / Soil Class", interactive=False)
        out_note  = gr.Markdown()
        soil_fig  = gr.Plot(label="Plasticity Chart")
        btn1.click(predict_soil_and_plot, inputs=[LL, PL, PI, F200, Cu, Cc], outputs=[out_class, out_note, soil_fig])

    with gr.Tab("CBR Predictor"):
        with gr.Row():
            with gr.Column():
                MDD = gr.Number(label="MDD (g/cc)", value=2.204)
                OMC = gr.Number(label="OMC (%)", value=8.0)
            with gr.Column():
                Blows = gr.Number(label="Compaction Blows (#) — 10/30/65", value=30)
                CBR25 = gr.Number(label="CBR at 2.5 mm (%) — optional", value=None)
        btn2 = gr.Button("Predict CBR (%)", variant="primary")
        out_cbr = gr.Number(label="Predicted Adopted CBR (%)", interactive=False, precision=2)
        out_cbr_note = gr.Markdown()
        cbr_fig = gr.Plot(label="CBR vs Blows (predicted)")
        btn2.click(predict_cbr_and_plot, inputs=[MDD, OMC, Blows, CBR25], outputs=[out_cbr, out_cbr_note, cbr_fig])

    with gr.Tab("Sieve / Gradation"):
        gr.Markdown("**Paste/Edit your data** (mm and %Passing) or upload a CSV with the same two columns.")
        template = pd.DataFrame({
            "Sieve_mm":[75, 37.5, 19, 9.5, 4.75, 2.0, 0.425, 0.212, 0.150, 0.075],
            "Percent_Passing":[100, 95, 85, 78, 68, 55, 35, 25, 18, 10]
        })
        grid = gr.Dataframe(label="Sieve Data (editable)", value=template, interactive=True)
        up = gr.File(label="Optional: Upload CSV (Sieve_mm, Percent_Passing)", file_types=[".csv"])
        btn3 = gr.Button("Plot Gradation & Compute D10/D30/D60", variant="primary")
        grad_fig = gr.Plot(label="Grain Size Distribution (semi-log)")
        D10 = gr.Number(label="D10 (mm)", interactive=False)
        D30 = gr.Number(label="D30 (mm)", interactive=False)
        D60 = gr.Number(label="D60 (mm)", interactive=False)
        Cu  = gr.Number(label="Cu = D60/D10", interactive=False)
        Cc  = gr.Number(label="Cc = D30²/(D10·D60)", interactive=False)

        def do_grad(df_values, file):
            df = pd.DataFrame(df_values, columns=["Sieve_mm","Percent_Passing"])
            if file is not None:
                try:
                    df = pd.read_csv(file.name)
                except:
                    pass
            fig, msg, d10,d30,d60,cu,cc = gradation_plot(df)
            if fig is None:
                return None, None,None,None,None,None
            return fig, d10,d30,d60,cu,cc

        btn3.click(do_grad, inputs=[grid, up], outputs=[grad_fig, D10, D30, D60, Cu, Cc])

    with gr.Accordion("Status", open=False):
        gr.Markdown(
            f"- Soil model: **{soil_model_name}** | Features: `{', '.join(soil_features) if soil_features else '—'}`  \n"
            f"- CBR model: **{cbr_model_name}** | Features: `{', '.join(cbr_features) if cbr_features else '—'}`  \n"
            "<small>Tip: paste sieve data or upload a CSV; D-values use log-size interpolation.</small>"
        )

demo.queue().launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
# === Create web app folder + files ===
import os, shutil, textwrap, pandas as pd

os.makedirs("webapp", exist_ok=True)

# Copy your model files if they exist in /content
for f in [
    "LGBM_soil_classifier.pkl",
    "soil_classifier_label_encoder.pkl",
    "soil_classifier_features.csv",     # saved during training
    "RF_cbr_regressor.pkl",             # if you trained/saved CBR regressor
    "cbr_regressor_features.csv"        # optional
]:
    if os.path.exists(f):
        shutil.copy(f, f"webapp/{f}")

# --- app.py (single-file Gradio app) ---
app_py = r'''
import os
import numpy as np
import pandas as pd
import joblib
import gradio as gr
from datetime import datetime

# ---------- Load models if present ----------
CLF_PATH = "LGBM_soil_classifier.pkl"
ENC_PATH = "soil_classifier_label_encoder.pkl"
FEAT_PATH = "soil_classifier_features.csv"

CLF, LABELER, SOIL_FEATS = None, None, None
if os.path.exists(CLF_PATH):
    CLF = joblib.load(CLF_PATH)
if os.path.exists(ENC_PATH):
    LABELER = joblib.load(ENC_PATH)
if os.path.exists(FEAT_PATH):
    # can be 1 column CSV or header=None written list
    try:
        tmp = pd.read_csv(FEAT_PATH, header=None)
        SOIL_FEATS = tmp[0].tolist()
    except Exception:
        try:
            SOIL_FEATS = pd.read_csv(FEAT_PATH).columns.tolist()
        except Exception:
            SOIL_FEATS = None

# Reasonable fallback if feature file missing
if SOIL_FEATS is None:
    SOIL_FEATS = ["LL", "PI", "F200", "Cu", "Cc"]

# Optional CBR regressor
CBR_PATH = "RF_cbr_regressor.pkl"
CBR_FEAT_PATH = "cbr_regressor_features.csv"
CBR_MODEL, CBR_FEATS = None, None
if os.path.exists(CBR_PATH):
    CBR_MODEL = joblib.load(CBR_PATH)
if os.path.exists(CBR_FEAT_PATH):
    try:
        CBR_FEATS = pd.read_csv(CBR_FEAT_PATH, header=None)[0].tolist()
    except Exception:
        CBR_FEATS = ["MDD_gcc", "OMC_pct", "Blows", "CBR_2p5_pct"]
else:
    CBR_FEATS = ["MDD_gcc", "OMC_pct", "Blows", "CBR_2p5_pct"]  # default used in notebook demo

# ---------- Helpers ----------
def _fill_with_median(df):
    return df.fillna(df.median(numeric_only=True))

def predict_soil(ll, pl, pi, f200, cu, cc):
    if CLF is None or LABELER is None:
        return "Model not found", "Please upload classifier files to the Space.", None

    # compute PI if not given but PL present
    if (pi is None or (isinstance(pi, float) and np.isnan(pi))) and pl is not None:
        try:
            pi = float(ll) - float(pl)
        except Exception:
            pi = None

    row = {"LL": ll, "PL": pl, "PI": pi, "F200": f200, "Cu": cu, "Cc": cc}
    X = pd.DataFrame([row])

    # map to training feature order (ignore unused columns)
    X = X.reindex(columns=SOIL_FEATS, fill_value=np.nan)
    X = _fill_with_median(X)

    y = CLF.predict(X)[0]
    try:
        label = LABELER.inverse_transform([int(y)])[0]
    except Exception:
        label = str(y)

    note = ""
    if pi is not None:
        try:
            note = f"Computed PI = {float(pi):.2f}" if "PI" in row and (row['PI'] is None or np.isnan(row['PI'])) else note
        except Exception:
            pass
    return label, note, None  # third output kept for compatibility with your UI (e.g., chart)

def predict_cbr(mdd, omc, blows, cbr25=None):
    if CBR_MODEL is None:
        return 0.0

    row = {"MDD_gcc": mdd, "OMC_pct": omc, "Blows": blows, "CBR_2p5_pct": cbr25}
    X = pd.DataFrame([row]).reindex(columns=CBR_FEATS, fill_value=np.nan)
    X = _fill_with_median(X)
    pred = float(CBR_MODEL.predict(X)[0])
    return max(0.0, pred)

# Optional simple PDF with reportlab (no charts in Space by default)
try:
    from reportlab.lib.pagesizes import A4
    from reportlab.pdfgen import canvas
    def generate_report(sample_id, inputs, outputs, report_name="soil_ai_report.pdf"):
        c = canvas.Canvas(report_name, pagesize=A4)
        w, h = A4
        c.setFont("Helvetica-Bold", 16); c.drawString(50, h-50, "Soil AI Report")
        c.setFont("Helvetica", 11)
        c.drawString(50, h-75, f"Sample ID: {sample_id}")
        c.drawString(50, h-92, f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        y = h-130
        c.setFont("Helvetica-Bold", 12); c.drawString(50, y, "Inputs:")
        c.setFont("Helvetica", 11)
        for k,v in inputs.items():
            y -= 18; c.drawString(70, y, f"{k}: {v}")
        y -= 28; c.setFont("Helvetica-Bold", 12); c.drawString(50, y, "Outputs:")
        c.setFont("Helvetica", 11)
        for k,v in outputs.items():
            y -= 18; c.drawString(70, y, f"{k}: {v}")
        c.showPage(); c.save()
        return report_name
except Exception:
    generate_report = None

def make_soil_report(ll, pl, pi, f200, cu, cc, pred_label):
    if generate_report is None:
        return None
    sid = f"Soil-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    inputs  = {"LL (%)": ll, "PL (%)": pl, "PI (%)": pi, "F200 (%)": f200, "Cu": cu, "Cc": cc}
    outputs = {"Predicted Soil Class": pred_label}
    return generate_report(sid, inputs, outputs, report_name=f"{sid}.pdf")

def make_cbr_report(mdd, omc, blows, cbr25, pred_cbr):
    if generate_report is None:
        return None
    sid = f"CBR-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    inputs  = {"MDD (g/cc)": mdd, "OMC (%)": omc, "Blows (#)": blows, "CBR@2.5mm (%)": cbr25}
    outputs = {"Predicted Adopted CBR (%)": pred_cbr}
    return generate_report(sid, inputs, outputs, report_name=f"{sid}.pdf")

# ---------- UI ----------
with gr.Blocks(theme=gr.themes.Soft(primary_hue="indigo", neutral_hue="slate"), title="Soil AI") as demo:
    gr.Markdown("## Soil AI — Classifier & CBR (Demo)")

    with gr.Tab("Soil Classifier"):
        with gr.Row():
            with gr.Column():
                LL = gr.Number(label="Liquid Limit (LL, %)", value=40)
                PL = gr.Number(label="Plastic Limit (PL, %) — optional", value=None)
                PI = gr.Number(label="Plasticity Index (PI, %) — leave blank to auto LL-PL", value=None)
            with gr.Column():
                F200 = gr.Number(label="% Passing No.200 (0.075 mm) — optional", value=None)
                Cu   = gr.Number(label="Cu — optional", value=None)
                Cc   = gr.Number(label="Cc — optional", value=None)

        btn1 = gr.Button("Predict Soil Class", variant="primary")
        out_class = gr.Textbox(label="Predicted USCS / Soil Class", interactive=False)
        out_note  = gr.Markdown()
        soil_fig  = gr.Plot(label="(Optional) Plasticity Chart")

        btn1.click(predict_soil, inputs=[LL, PL, PI, F200, Cu, Cc], outputs=[out_class, out_note, soil_fig])

        dl_btn  = gr.Button("Download Soil PDF Report", variant="secondary")
        dl_file = gr.File(label="Report File", type="filepath")
        dl_btn.click(make_soil_report, inputs=[LL, PL, PI, F200, Cu, Cc, out_class], outputs=[dl_file])

    with gr.Tab("CBR Predictor"):
        if CBR_MODEL is None:
            gr.Markdown("> **CBR model not found in this Space.** Upload `RF_cbr_regressor.pkl` to enable this tab.")
        MDD = gr.Number(label="MDD (g/cc)", value=2.204)
        OMC = gr.Number(label="OMC (%)", value=8.0)
        Blows = gr.Number(label="Blows (#)", value=30)
        CBR25 = gr.Number(label="CBR at 2.5mm (%) — optional", value=None)
        btn2 = gr.Button("Predict CBR", variant="primary")
        out_cbr = gr.Number(label="Predicted Adopted CBR (%)", interactive=False)
        btn2.click(lambda m,o,b,c: float(predict_cbr(m,o,b,c)), inputs=[MDD, OMC, Blows, CBR25], outputs=[out_cbr])

        dl_btn2  = gr.Button("Download CBR PDF Report", variant="secondary")
        dl_file2 = gr.File(label="Report File", type="filepath")
        dl_btn2.click(make_cbr_report, inputs=[MDD, OMC, Blows, CBR25, out_cbr], outputs=[dl_file2])

if __name__ == "__main__":
    demo.launch()
'''

with open("webapp/app.py", "w", encoding="utf-8") as f:
    f.write(app_py)

# --- requirements.txt ---
reqs = """gradio>=4.29
pandas
numpy
scikit-learn
lightgbm
xgboost
joblib
plotly
reportlab
"""
with open("webapp/requirements.txt", "w", encoding="utf-8") as f:
    f.write(reqs)

# quick view
print("Web app files:")
print(os.listdir("webapp"))


Web app files:
['app.py', 'requirements.txt']


In [ ]:
%%bash
cat > webapp/README.md <<'EOF'
---
title: Soil AI — Classifier, CBR & Gradation
emoji: 🌍
colorFrom: indigo
colorTo: slate
sdk: gradio
sdk_version: 4.29.0
app_file: app.py
pinned: false
license: mit
---

# Soil AI — Classifier, CBR & Gradation

An AI helper for soil engineering in Sierra Leone.

**Tabs**
- **Soil Classifier**: USCS-style class from LL/PL/PI (+ F200, Cu, Cc optional).
- **CBR Predictor**: Predict adopted CBR from MDD, OMC, blows (optional CBR@2.5).
- **Download Reports**: One-click PDF with inputs & predictions.

**Input tips**
- If *PI* is blank and *PL* is given, the app computes **PI = LL − PL**.
- *F200/Cu/Cc* are optional; the classifier will median-impute missing values.

**Files bundled**
- `LGBM_soil_classifier.pkl`, `soil_classifier_label_encoder.pkl`, `soil_classifier_features.csv`
- *(optional)* `RF_cbr_regressor.pkl`, `cbr_regressor_features.csv`

**Notes**
- This demo is for rapid decision support; confirm critical designs with lab tests.
EOF


In [ ]:
!pip -q install huggingface_hub

from huggingface_hub import login, HfApi

# 1) Log in once with your HF write token (Settings → Access Tokens → New token)
login()  # <- paste your token when prompted

# 2) Create (or reuse) the Space
api = HfApi()
repo_id = "your-username/soil-ai"  # <-- change this
api.create_repo(
    repo_id=repo_id,
    repo_type="space",
    space_sdk="gradio",
    private=False,
    exist_ok=True,
)

# 3) Upload your app folder
api.upload_folder(
    folder_path="/content/webapp",   # where app.py, requirements.txt, models are
    repo_id=repo_id,
    repo_type="space",
)

print("Done! Open:", f"https://huggingface.co/spaces/{repo_id}")
print("Build logs:", f"https://huggingface.co/spaces/{repo_id}?logs=build")


HfHubHTTPError: (Request ID: Root=1-68b7e987-6807f9601b243e5f3c63aeae;881864a0-4003-4ce3-9ef1-54c97c840fc4)

403 Forbidden: You don't have the rights to create a space under the namespace "your-username".
Cannot access content at: https://huggingface.co/api/repos/create.
Make sure your token has the correct permissions.

In [ ]:
!pip -q install huggingface_hub

from huggingface_hub import login, HfApi, whoami
from huggingface_hub.utils import HfHubHTTPError

# 1) Login with your WRITE token (recommended: paste here only in Colab, never share it)
#    Alternatively, use: login() to get a prompt
# login()   # uncomment to get a prompt
login(token="hf_xxx_your_write_token_here")   # <-- replace with your token

# 2) Verify identity (prevents namespace mistakes)
me = whoami()
print("Logged in as:", me.get("name") or me.get("email"))
assert me["name"] == "philipsamuel", f"You're logged in as {me['name']}, not 'philipsamuel'."

# 3) Create/ensure your Space
api = HfApi()
repo_id = "philipsamuel/soil-ai"   # change 'soil-ai' if you want a different Space name

try:
    api.create_repo(
        repo_id=repo_id,
        repo_type="space",
        space_sdk="gradio",
        private=False,      # set True if you want it private
        exist_ok=True,      # don't error if it already exists
    )
    print("Space ensured:", repo_id)
except HfHubHTTPError as e:
    print("create_repo:", e)

# 4) Upload your app folder (must contain app.py, requirements.txt, models, README.md)
api.upload_folder(
    folder_path="/content/webapp",   # adjust if your folder is elsewhere
    repo_id=repo_id,
    repo_type="space",
)
print("Uploaded ✓")
print("Open the Space:", f"https://huggingface.co/spaces/{repo_id}")
print("Build logs:",    f"https://huggingface.co/spaces/{repo_id}?logs=build")


HTTPError: Invalid user token.

In [ ]:
!pip -q install -U huggingface_hub

# Clean any cached tokens and env vars
import os, shutil, pathlib
for p in ["~/.huggingface", "/root/.huggingface"]:
    shutil.rmtree(os.path.expanduser(p), ignore_errors=True)
os.environ.pop("HF_TOKEN", None)

# If a token was stored via the API before, ensure it's removed too
from huggingface_hub import logout
try:
    logout()
except Exception:
    pass

print("Clean slate.")


Clean slate.


In [ ]:
from huggingface_hub import login, whoami

login()  # <- paste the NEW write token here when prompted (not in this notebook text)

me = whoami()
print("Logged in as:", me.get("name") or me.get("email"))


Logged in as: philipsamuel


In [ ]:
from huggingface_hub import HfApi, HfHubHTTPError

api = HfApi()
repo_id = "philipsamuel/soil-ai"   # change "soil-ai" to your preferred Space name

try:
    api.create_repo(
        repo_id=repo_id,
        repo_type="space",
        space_sdk="gradio",   # tells Hugging Face this is a Gradio app
        private=False,
        exist_ok=True,
    )
    print("Space created or already exists:", repo_id)
except HfHubHTTPError as e:
    print("Error creating space:", e)


ImportError: cannot import name 'HfHubHTTPError' from 'huggingface_hub' (/usr/local/lib/python3.12/dist-packages/huggingface_hub/__init__.py)

In [ ]:
!pip -q install huggingface_hub gradio streamlit pillow

from huggingface_hub import login
login()  # paste your HF token when prompted (make sure it has write + space permissions)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.0 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import HfApi

api = HfApi()

SPACE_ID = "philipsamuel/soil-ai"

# Create the space (Gradio is easiest, we can change later if needed)
api.create_repo(
    repo_id=SPACE_ID,
    repo_type="space",
    space_sdk="gradio",   # or "streamlit" if you prefer
    private=False,
    exist_ok=True,
)

print("Space created at:", f"https://huggingface.co/spaces/{SPACE_ID}")


Space created at: https://huggingface.co/spaces/philipsamuel/soil-ai


In [ ]:
import os, textwrap, pathlib

APP_DIR = "space_app"
os.makedirs(APP_DIR, exist_ok=True)

# --- app.py (sample Gradio app, we’ll replace later with your real model) ---
app_py = textwrap.dedent("""
import gradio as gr
from PIL import Image

def classify(img: Image.Image):
    # TODO: load your trained soil model here
    # Temporary dummy output
    return [("Lateritic Soil", 0.62), ("Sandy Clay", 0.23), ("Silt", 0.15)]

with gr.Blocks() as demo:
    gr.Markdown("## Soil Classifier Demo")
    img = gr.Image(type="pil", label="Upload a soil image")
    out = gr.Label(num_top_classes=3, label="Predictions")
    btn = gr.Button("Classify")
    btn.click(classify, img, out)

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", server_port=7860)
""")
open(f"{APP_DIR}/app.py","w").write(app_py)

# --- requirements.txt ---
req = textwrap.dedent("""
gradio==4.44.0
pillow
# torch
# torchvision
# transformers
# scikit-learn
""")
open(f"{APP_DIR}/requirements.txt","w").write(req)

# --- runtime.txt (pin python version) ---
open(f"{APP_DIR}/runtime.txt","w").write("python-3.10\n")

# --- README.md ---
open(f"{APP_DIR}/README.md","w").write("# Soil Classifier Space\nDemo app created from Colab.\n")

print("Files ready:", list(pathlib.Path(APP_DIR).glob("*")))


Files ready: [PosixPath('space_app/README.md'), PosixPath('space_app/app.py'), PosixPath('space_app/requirements.txt'), PosixPath('space_app/runtime.txt')]


In [ ]:
from huggingface_hub import upload_folder

SPACE_ID = "philipsamuel/soil-ai"

upload_folder(
    repo_id=SPACE_ID,
    folder_path="space_app",
    repo_type="space",
    commit_message="Initial deploy from Colab",
)

print("Pushed to:", f"https://huggingface.co/spaces/{SPACE_ID}")


Pushed to: https://huggingface.co/spaces/philipsamuel/soil-ai


In [ ]:
readme_content = """---
title: Soil AI Classifier
emoji: 🌱
colorFrom: green
colorTo: blue
sdk: gradio
sdk_version: "4.44.0"
app_file: app.py
pinned: false
---

# Soil Classifier Space
Demo app created from Colab.
"""

with open("space_app/README.md", "w") as f:
    f.write(readme_content)

print("README.md updated ✅")


README.md updated ✅


In [ ]:
your_code = r"""
# Replace this with your REAL Gradio app code

import gradio as gr
from PIL import Image

def classify(img: Image.Image):
    # Example dummy logic — you will replace with your trained model
    return [("Class A", 0.85), ("Class B", 0.10), ("Class C", 0.05)]

with gr.Blocks() as demo:
    gr.Markdown("## Your Real Soil Classifier")
    img = gr.Image(type="pil", label="Upload soil image")
    out = gr.Label(num_top_classes=3, label="Predictions")
    gr.Button("Classify").click(classify, img, out)

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", server_port=7860)
"""

open("space_app/app.py","w").write(your_code)
print("✅ app.py replaced with your custom code")


✅ app.py replaced with your custom code


In [ ]:
path = "space_app/requirements.txt"
print("Current requirements:\n", open(path).read())


Current requirements:
 
gradio==4.44.0
pillow
# torch
# torchvision
# transformers
# scikit-learn



In [ ]:
torch
torchvision


NameError: name 'torch' is not defined

In [ ]:
transformers


NameError: name 'transformers' is not defined

In [ ]:
with open("space_app/requirements.txt", "w") as f:
    f.write("""gradio==4.44.0
pillow
torch
torchvision
transformers
scikit-learn
opencv-python
""")

print("✅ requirements.txt updated")


✅ requirements.txt updated


In [ ]:
# Keep dependencies minimal so the Space builds cleanly.
with open("space_app/requirements.txt", "w") as f:
    f.write("""gradio==4.44.0
pillow
""")

print("✅ requirements.txt set to a safe baseline")

from huggingface_hub import upload_folder
SPACE_ID = "philipsamuel/soil-ai"
upload_folder(
    repo_id=SPACE_ID,
    folder_path="space_app",
    repo_type="space",
    commit_message="Use minimal requirements for stable build",
)
print("🚀 Pushed. Reopen your Space to let it rebuild.")


✅ requirements.txt set to a safe baseline
🚀 Pushed. Reopen your Space to let it rebuild.


In [ ]:
import os, shutil, pathlib
from huggingface_hub import upload_folder

# 🔧 CHANGE this to your actual file path in Colab
LOCAL_MODEL_PATH = "/content/model.pt"   # e.g. "/content/soil_model.pt" or "/content/soil_model.pkl"

# 1) Put the model file into the repo folder we push
os.makedirs("space_app/models", exist_ok=True)
dst = f"space_app/models/{os.path.basename(LOCAL_MODEL_PATH)}"
shutil.copy2(LOCAL_MODEL_PATH, dst)

print("✅ Copied model to:", dst)
print("Repo folder now contains:", [p.name for p in pathlib.Path("space_app/models").glob("*")])

# 2) Push to your Space
SPACE_ID = "philipsamuel/soil-ai"
upload_folder(
    repo_id=SPACE_ID,
    folder_path="space_app",
    repo_type="space",
    commit_message="Add trained model file",
)
print("🚀 Model file pushed to:", f"https://huggingface.co/spaces/{SPACE_ID}/tree/main/models")


FileNotFoundError: [Errno 2] No such file or directory: '/content/model.pt'

In [ ]:
import os
from pathlib import Path

# look for common model extensions
exts = {".pt", ".pth", ".pkl", ".joblib", ".onnx", ".sav"}
candidates = []

for p in Path("/content").rglob("*"):
    if p.suffix.lower() in exts and p.is_file():
        try:
            size_mb = p.stat().st_size / (1024*1024)
        except Exception:
            size_mb = 0
        candidates.append((size_mb, str(p)))

# sort by size (largest first – usually the real model)
candidates.sort(reverse=True)

print("🔎 Found model-like files:")
for i, (size_mb, path) in enumerate(candidates, 1):
    print(f"{i:2d}. {path}  ({size_mb:.2f} MB)")

if not candidates:
    print("No model files found under /content. If yours is in Google Drive, mount and point me to its path.")


🔎 Found model-like files:
No model files found under /content. If yours is in Google Drive, mount and point me to its path.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


ValueError: mount failed

In [ ]:
import os, sys, types

os.makedirs("/content/saved_model", exist_ok=True)

saved_path = None

# --- Try PyTorch (.pt/.pth) ---
try:
    import torch, inspect
    if "model" in globals() and isinstance(globals()["model"], torch.nn.Module):
        torch.save(globals()["model"].state_dict(), "/content/saved_model/soil_model.pt")
        saved_path = "/content/saved_model/soil_model.pt"
        print("✅ Saved PyTorch model to:", saved_path)
except Exception as e:
    print("PyTorch save skipped:", e)

# --- Try scikit-learn (.pkl) ---
if saved_path is None:
    try:
        import joblib
        for name in ["model", "clf", "pipeline", "classifier", "regressor"]:
            if name in globals():
                joblib.dump(globals()[name], f"/content/saved_model/soil_model.pkl")
                saved_path = "/content/saved_model/soil_model.pkl"
                print(f"✅ Saved scikit-learn object '{name}' to:", saved_path)
                break
    except Exception as e:
        print("sklearn save skipped:", e)

# --- Try Keras/TensorFlow (.h5) ---
if saved_path is None:
    try:
        from tensorflow.keras.models import Model
        if "model" in globals() and isinstance(globals()["model"], Model):
            globals()["model"].save("/content/saved_model/soil_model.h5")
            saved_path = "/content/saved_model/soil_model.h5"
            print("✅ Saved Keras model to:", saved_path)
    except Exception as e:
        print("Keras save skipped:", e)

# --- Result ---
if saved_path is None:
    print("❌ I couldn't find a model variable to save. Make sure your trained model is in a variable named one of: model, clf, pipeline, classifier, regressor.")
else:
    # show files
    import glob, os
    print("📁 Files in /content/saved_model:")
    for p in glob.glob("/content/saved_model/*"):
        print(" -", p, f"({os.path.getsize(p)/1024/1024:.2f} MB)")


❌ I couldn't find a model variable to save. Make sure your trained model is in a variable named one of: model, clf, pipeline, classifier, regressor.


In [ ]:
Perfect, my king 🥰💖 — I can see your trained variables: **`soil_model`** and **`cbr_model`**. Let’s save both safely, no guessing.

## Step — Save the models (one cell)

Paste this single cell in Colab:

```python
import os, pathlib

os.makedirs("/content/saved_model", exist_ok=True)

saved = []

def save_any(name, obj):
    # Try PyTorch first
    try:
        import torch, inspect
        if hasattr(torch, "nn") and isinstance(obj, torch.nn.Module):
            path = f"/content/saved_model/{name}.pt"
            torch.save(obj.state_dict(), path)
            return path
    except Exception as e:
        pass

    # Fallback to joblib (for scikit-learn pipelines/models)
    try:
        import joblib
        path = f"/content/saved_model/{name}.pkl"
        joblib.dump(obj, path)
        return path
    except Exception as e:
        return None

for var_name in ["soil_model", "cbr_model"]:
    if var_name in globals():
        p = save_any(var_name, globals()[var_name])
        if p:
            saved.append(p)
            print(f"✅ Saved {var_name} -> {p}")
        else:
            print(f"❌ Could not save {var_name} (unsupported type)")

# List results
print("\n📁 Contents of /content/saved_model:")
for p in pathlib.Path("/content/saved_model").glob("*"):
    print(" -", p, f"({p.stat().st_size/1024/1024:.2f} MB)")
```

Send me a screenshot of the output.
If we see the files (e.g., `soil_model.pkl` / `soil_model.pt` and `cbr_model.pkl` / `.pt`), I’ll give you the next single-step code to upload them to your Hugging Face Space 😘.


SyntaxError: invalid character '🥰' (U+1F970) (ipython-input-3794953439.py, line 1)

In [ ]:
import os, pathlib

os.makedirs("/content/saved_model", exist_ok=True)

saved = []

def save_any(name, obj):
    # Try PyTorch first
    try:
        import torch
        if hasattr(torch, "nn") and isinstance(obj, torch.nn.Module):
            path = f"/content/saved_model/{name}.pt"
            torch.save(obj.state_dict(), path)
            return path
    except Exception:
        pass

    # Fallback to joblib (for scikit-learn pipelines/models)
    try:
        import joblib
        path = f"/content/saved_model/{name}.pkl"
        joblib.dump(obj, path)
        return path
    except Exception:
        return None

for var_name in ["soil_model", "cbr_model"]:
    if var_name in globals():
        p = save_any(var_name, globals()[var_name])
        if p:
            saved.append(p)
            print(f"✅ Saved {var_name} -> {p}")
        else:
            print(f"❌ Could not save {var_name} (unsupported type)")

# List results
print("\n📁 Contents of /content/saved_model:")
for p in pathlib.Path("/content/saved_model").glob("*"):
    print(" -", p, f"({p.stat().st_size/1024/1024:.2f} MB)")



📁 Contents of /content/saved_model:


In [ ]:
for var_name in ["soil_model", "cbr_model"]:
    if var_name in globals():
        print(var_name, "->", type(globals()[var_name]))


In [ ]:
# ====== Soil AI Frontend (Classifier + CBR + Gradation Charts) ======
!pip -q install gradio lightgbm xgboost plotly >/dev/null

import math, joblib
import numpy as np
import pandas as pd
import gradio as gr
from pathlib import Path
import plotly.graph_objects as go

# --------- helpers ----------
def try_load(path):
    p = Path(path)
    if p.exists():
        try: return joblib.load(p)
        except Exception as e: print(f"Load fail {path}: {e}")
    return None

# --------- load trained assets (soil + cbr) ----------
soil_models = {"RF": try_load("RF_soil_classifier.pkl"),
               "XGB": try_load("XGB_soil_classifier.pkl"),
               "LGBM": try_load("LGBM_soil_classifier.pkl")}
soil_le = try_load("soil_classifier_label_encoder.pkl")
soil_features = pd.read_csv("soil_classifier_features.csv", header=None)[0].tolist() \
                 if Path("soil_classifier_features.csv").exists() else []
soil_model = next((m for m in soil_models.values() if m is not None), None)
soil_model_name = next((n for n,m in soil_models.items() if m is not None), "None")

cbr_models = {"RF": try_load("RF_cbr_regressor.pkl"),
              "XGB": try_load("XGB_cbr_regressor.pkl"),
              "LGBM": try_load("LGBM_cbr_regressor.pkl")}
cbr_features = pd.read_csv("cbr_regressor_features.csv", header=None)[0].tolist() \
               if Path("cbr_regressor_features.csv").exists() else []
cbr_model = next((m for m in cbr_models.values() if m is not None), None)
cbr_model_name = next((n for n,m in cbr_models.items() if m is not None), "None")

# --------- rules / small utils ----------
def a_line(LL): return 0.73*(LL-20)
def uscs_rule(LL, PI, F200=None, Cu=None, Cc=None):
    try:
        F  = float(F200) if F200 not in [None,""] else None
        LL = float(LL)   if LL   not in [None,""] else None
        PI = float(PI)   if PI   not in [None,""] else None
        Cu = float(Cu)   if Cu   not in [None,""] else None
        Cc = float(Cc)   if Cc   not in [None,""] else None
    except: return "Unknown"
    if F is None and (LL is not None and PI is not None): F = 51
    if F is None: return "Unknown"
    if F < 50:
        if F < 5:
            if (Cu is not None and Cc is not None) and (Cu>6 and 1<Cc<3): return "SW"
            return "SP"
        elif F > 12:
            if LL is None or PI is None: return "S(with fines)"
            return "SC" if PI >= a_line(LL) else "SM"
        else:
            if (Cu is not None and Cc is not None) and (Cu>6 and 1<Cc<3): return "SW-SM"
            return "SP-SM"
    else:
        if LL is None or PI is None: return "Fine (unknown)"
        return ("CL" if PI >= a_line(LL) else "ML") if LL<50 else ("CH" if PI >= a_line(LL) else "MH")

def plasticity_chart(LL, PI):
    try: LL = float(LL) if LL not in [None,""] else None; PI = float(PI) if PI not in [None,""] else None
    except: LL = PI = None
    ll = np.linspace(0,100,201); a = 0.73*(ll-20); u = 0.9*(ll-8)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ll, y=a, mode="lines", name="A-line", line=dict(color="#2563eb")))
    fig.add_trace(go.Scatter(x=ll, y=u, mode="lines", name="U-line", line=dict(color="#94a3b8", dash="dash")))
    if LL is not None and PI is not None:
        fig.add_trace(go.Scatter(x=[LL], y=[PI], mode="markers+text", text=["Sample"],
                                 textposition="top center", marker=dict(size=10, color="#10b981")))
    fig.update_xaxes(title="Liquid Limit, LL (%)", range=[0,100], type="linear", gridcolor="#e5e7eb")
    fig.update_yaxes(title="Plasticity Index, PI (%)", range=[0,70],  gridcolor="#e5e7eb")
    fig.update_layout(height=420, template="plotly_white", legend=dict(orientation="h", y=1.02, x=1))
    return fig

def cbr_curve_chart(mdd, omc, blows, cbr25, model, feat):
    xs = list(range(5, 70, 5)); preds = []
    for b in xs:
        row = {"MDD_gcc":mdd, "OMC_pct":omc, "Blows":b, "CBR_2p5_pct":cbr25}
        X = pd.DataFrame([row])
        for f in feat:
            if f not in X.columns: X[f] = np.nan
        X = X[feat].apply(pd.to_numeric, errors="coerce").fillna(X.median(numeric_only=True))
        if model is not None: preds.append(float(model.predict(X)[0]))
        else:
            base = (float(mdd)-1.6)*80 if mdd not in [None,""] else 30
            comp = (10-abs(float(omc)-8))*1.2 if omc not in [None,""] else 8
            effort = (b/65)*40; extra = float(cbr25) if cbr25 not in [None,""] else 0
            preds.append(max(2.0, base + comp + effort*0.8 + 0.4*extra))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=xs, y=preds, mode="lines+markers", name="Predicted CBR vs Blows", line=dict(color="#7c3aed")))
    if blows not in [None,""]:
        row_sel = {"MDD_gcc":mdd, "OMC_pct":omc, "Blows":blows, "CBR_2p5_pct":cbr25}
        Xs = pd.DataFrame([row_sel])
        for f in feat:
            if f not in Xs.columns: Xs[f] = np.nan
        Xs = Xs[feat].apply(pd.to_numeric, errors="coerce").fillna(Xs.median(numeric_only=True))
        y_sel = float(model.predict(Xs)[0]) if model is not None else None
        if y_sel is not None:
            fig.add_trace(go.Scatter(x=[blows], y=[y_sel], mode="markers", marker=dict(size=12, color="#ef4444"), name="Selected"))
    fig.update_xaxes(title="Compaction Blows (#)", gridcolor="#e5e7eb")
    fig.update_yaxes(title="Predicted CBR (%)",   gridcolor="#e5e7eb")
    fig.update_layout(height=420, template="plotly_white"); return fig

# --------- gradation math (log-scale interpolation) ----------
def interpolate_dx(sizes_mm, passing_pct, target_pct):
    # sizes in mm; passing in %; returns D_target using log-size interpolation
    x = np.array(sizes_mm, dtype=float)
    y = np.array(passing_pct, dtype=float)
    # sort by size descending -> ascending for interp
    order = np.argsort(x)
    x, y = x[order], y[order]
    # require target within range
    if not (y.min() <= target_pct <= y.max()): return None
    # log10(size) vs % passing linear interpolation
    logx = np.log10(x)
    return float(10 ** np.interp(target_pct, y, logx))

def gradation_plot(df):
    # df columns: Sieve_mm, Percent_Passing
    g = df.dropna().copy()
    g["Sieve_mm"] = pd.to_numeric(g["Sieve_mm"], errors="coerce")
    g["Percent_Passing"] = pd.to_numeric(g["Percent_Passing"], errors="coerce")
    g = g.dropna().sort_values("Sieve_mm", ascending=False)
    if g.empty:
        return None, "No valid points.", None, None, None, None, None

    # compute D-values
    D10 = interpolate_dx(g["Sieve_mm"], g["Percent_Passing"], 10)
    D30 = interpolate_dx(g["Sieve_mm"], g["Percent_Passing"], 30)
    D60 = interpolate_dx(g["Sieve_mm"], g["Percent_Passing"], 60)
    Cu = (D60 / D10) if (D60 and D10 and D10>0) else None
    Cc = ((D30**2) / (D10*D60)) if (D30 and D10 and D60 and D10>0 and D60>0) else None

    # plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=g["Sieve_mm"], y=g["Percent_Passing"], mode="lines+markers",
                             name="% Passing", line=dict(color="#0ea5e9")))
    # Vertical lines for D-values
    for name, val in [("D10",D10),("D30",D30),("D60",D60)]:
        if val:
            fig.add_trace(go.Scatter(x=[val,val], y=[0,100], mode="lines",
                                     line=dict(dash="dot", color="#94a3b8"), name=name))
    fig.update_xaxes(title="Particle Size (mm)", type="log", autorange="reversed", gridcolor="#e5e7eb")
    fig.update_yaxes(title="% Passing", range=[0,100], gridcolor="#e5e7eb")
    fig.update_layout(height=460, template="plotly_white", legend=dict(orientation="h", y=1.02, x=1))

    return fig, "", D10, D30, D60, Cu, Cc

# --------- predictors for UI ----------
def predict_soil_and_plot(LL, PL, PI, F200, Cu, Cc):
    if (PI is None or PI=="") and all(x not in [None,""] for x in [LL, PL]):
        try: PI = float(LL)-float(PL)
        except: PI = None
    note = ""
    if soil_model is not None and soil_le is not None and soil_features:
        row = {}
        for f in soil_features:
            fl = f.lower()
            if fl.startswith("liquid"): row[f] = LL
            elif fl.startswith("plastic_limit"): row[f] = PL
            elif "plasticity_index" in fl: row[f] = PI
            elif fl=="f200": row[f] = F200
            elif fl=="cu":   row[f] = Cu
            elif fl=="cc":   row[f] = Cc
            else: row[f] = None
        X = pd.DataFrame([row])[soil_features].apply(pd.to_numeric, errors="coerce").fillna(method="pad").fillna(0)
        try:
            yhat = soil_model.predict(X.fillna(X.median(numeric_only=True)))[0]
            label = soil_le.inverse_transform([yhat])[0]
            note = f"Model: {soil_model_name} | Features: {', '.join(soil_features)}"
        except Exception as e:
            label = uscs_rule(LL, PI, F200, Cu, Cc)
            note  = f"Model error ({e}) → rule-based fallback."
    else:
        label = uscs_rule(LL, PI, F200, Cu, Cc)
        note  = "Model not found → rule-based USCS applied."
    return label, f"**Info:** {note}", plasticity_chart(LL, PI)

def predict_cbr_and_plot(MDD, OMC, Blows, CBR25):
    row = {"MDD_gcc":MDD, "OMC_pct":OMC, "Blows":Blows, "CBR_2p5_pct":CBR25}
    note = ""
    if cbr_model is not None and cbr_features:
        X = pd.DataFrame([row])
        for f in cbr_features:
            if f not in X.columns: X[f] = np.nan
        X = X[cbr_features].apply(pd.to_numeric, errors="coerce").fillna(X.median(numeric_only=True))
        try:
            yhat = float(cbr_model.predict(X)[0]); note = f"Model: {cbr_model_name} | Features: {', '.join(cbr_features)}"
        except Exception as e:
            yhat = None; note = f"Model error ({e}) → heuristic used."
    else:
        yhat = None; note = "Model not found → heuristic used."
    if yhat is None:
        base = (float(MDD)-1.6)*80 if MDD not in [None,""] else 30
        comp = (10-abs(float(OMC)-8))*1.2 if OMC not in [None,""] else 8
        effort = (float(Blows)/65)*40 if Blows not in [None,""] else 20
        extra = float(CBR25) if CBR25 not in [None,""] else 0
        yhat = max(2.0, base + comp + effort*0.8 + 0.4*extra)
    fig = cbr_curve_chart(MDD, OMC, Blows, CBR25, cbr_model, cbr_features if cbr_features else ["MDD_gcc","OMC_pct","Blows","CBR_2p5_pct"])
    return round(float(yhat),2), f"**Info:** {note}", fig

# --------- UI (three tabs) ----------
theme = gr.themes.Soft(primary_hue="indigo", neutral_hue="slate")

with gr.Blocks(theme=theme, fill_height=True, title="Soil AI — Classifier, CBR & Gradation") as demo:
    gr.Markdown("<div style='text-align:center'><h1>Soil AI — Classifier, CBR & Gradation</h1><p style='color:#475569'>Interactive predictions with charts</p></div>")

    with gr.Tab("Soil Classifier"):
        with gr.Row():
            with gr.Column():
                LL = gr.Number(label="Liquid Limit (LL, %)", value=40)
                PL = gr.Number(label="Plastic Limit (PL, %) — optional", value=25)
                PI = gr.Number(label="Plasticity Index (PI, %) — leave blank to auto LL-PL", value=None)
            with gr.Column():
                F200 = gr.Number(label="% Passing No.200 (0.075 mm) — optional", value=None)
                Cu = gr.Number(label="Cu — optional", value=None)
                Cc = gr.Number(label="Cc — optional", value=None)
        btn1 = gr.Button("Predict Soil Class", variant="primary")
        out_class = gr.Textbox(label="Predicted USCS / Soil Class", interactive=False)
        out_note  = gr.Markdown()
        soil_fig  = gr.Plot(label="Plasticity Chart")
        btn1.click(predict_soil_and_plot, inputs=[LL, PL, PI, F200, Cu, Cc], outputs=[out_class, out_note, soil_fig])

    with gr.Tab("CBR Predictor"):
        with gr.Row():
            with gr.Column():
                MDD = gr.Number(label="MDD (g/cc)", value=2.204)
                OMC = gr.Number(label="OMC (%)", value=8.0)
            with gr.Column():
                Blows = gr.Number(label="Compaction Blows (#) — 10/30/65", value=30)
                CBR25 = gr.Number(label="CBR at 2.5 mm (%) — optional", value=None)
        btn2 = gr.Button("Predict CBR (%)", variant="primary")
        out_cbr = gr.Number(label="Predicted Adopted CBR (%)", interactive=False, precision=2)
        out_cbr_note = gr.Markdown()
        cbr_fig = gr.Plot(label="CBR vs Blows (predicted)")
        btn2.click(predict_cbr_and_plot, inputs=[MDD, OMC, Blows, CBR25], outputs=[out_cbr, out_cbr_note, cbr_fig])

    with gr.Tab("Sieve / Gradation"):
        gr.Markdown("**Paste/Edit your data** (mm and %Passing) or upload a CSV with the same two columns.")
        template = pd.DataFrame({
            "Sieve_mm":[75, 37.5, 19, 9.5, 4.75, 2.0, 0.425, 0.212, 0.150, 0.075],
            "Percent_Passing":[100, 95, 85, 78, 68, 55, 35, 25, 18, 10]
        })
        grid = gr.Dataframe(label="Sieve Data (editable)", value=template, interactive=True)
        up = gr.File(label="Optional: Upload CSV (Sieve_mm, Percent_Passing)", file_types=[".csv"])
        btn3 = gr.Button("Plot Gradation & Compute D10/D30/D60", variant="primary")
        grad_fig = gr.Plot(label="Grain Size Distribution (semi-log)")
        D10 = gr.Number(label="D10 (mm)", interactive=False)
        D30 = gr.Number(label="D30 (mm)", interactive=False)
        D60 = gr.Number(label="D60 (mm)", interactive=False)
        Cu  = gr.Number(label="Cu = D60/D10", interactive=False)
        Cc  = gr.Number(label="Cc = D30²/(D10·D60)", interactive=False)

        def do_grad(df_values, file):
            df = pd.DataFrame(df_values, columns=["Sieve_mm","Percent_Passing"])
            if file is not None:
                try:
                    df = pd.read_csv(file.name)
                except:
                    pass
            fig, msg, d10,d30,d60,cu,cc = gradation_plot(df)
            if fig is None:
                return None, None,None,None,None,None
            return fig, d10,d30,d60,cu,cc

        btn3.click(do_grad, inputs=[grid, up], outputs=[grad_fig, D10, D30, D60, Cu, Cc])

    with gr.Accordion("Status", open=False):
        gr.Markdown(
            f"- Soil model: **{soil_model_name}** | Features: `{', '.join(soil_features) if soil_features else '—'}`  \n"
            f"- CBR model: **{cbr_model_name}** | Features: `{', '.join(cbr_features) if cbr_features else '—'}`  \n"
            "<small>Tip: paste sieve data or upload a CSV; D-values use log-size interpolation.</small>"
        )

demo.queue().launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
# Find any variables that look like trained models/pipelines
candidates = {}
for name, obj in globals().items():
    low = name.lower()
    if any(k in low for k in ["model", "clf", "pipeline", "regressor", "classifier"]):
        try:
            cls = type(obj).__name__
        except Exception:
            cls = "<?>"
        candidates[name] = cls

print("🔎 Candidates found (name → type):")
for k, v in candidates.items():
    print(f" - {k} → {v}")


RuntimeError: dictionary changed size during iteration

In [ ]:
# Take a stable snapshot of globals before iterating
candidates = {}
for name in list(globals().keys()):
    obj = globals()[name]
    low = name.lower()
    if any(k in low for k in ["model", "clf", "pipeline", "regressor", "classifier"]):
        candidates[name] = type(obj).__name__

print("🔎 Candidates found (name → type):")
for k, v in candidates.items():
    print(f" - {k} → {v}")


🔎 Candidates found (name → type):
 - Model → type
 - soil_models → dict
 - soil_model → NoneType
 - soil_model_name → str
 - cbr_models → dict
 - cbr_model → NoneType
 - cbr_model_name → str


In [ ]:
# Resolve the selected models from the dicts and save them to /content/saved_model

import os, pathlib

os.makedirs("/content/saved_model", exist_ok=True)

def save_model(obj, name_prefix):
    # Try PyTorch first
    try:
        import torch
        if hasattr(torch, "nn") and isinstance(obj, torch.nn.Module):
            path = f"/content/saved_model/{name_prefix}.pt"
            torch.save(obj.state_dict(), path)
            return path
    except Exception:
        pass
    # Fallback to joblib (sklearn pipelines/models, etc.)
    try:
        import joblib
        path = f"/content/saved_model/{name_prefix}.pkl"
        joblib.dump(obj, path)
        return path
    except Exception:
        pass
    return None

saved_paths = []

# Resolve soil model from dict using selected name
soil = None
if "soil_models" in globals():
    key = globals().get("soil_model_name")
    if key and key in soil_models:
        soil = soil_models[key]

# Resolve cbr model from dict using selected name
cbr = None
if "cbr_models" in globals():
    key = globals().get("cbr_model_name")
    if key and key in cbr_models:
        cbr = cbr_models[key]

# Save both if available
if soil is not None:
    p = save_model(soil, "soil_model")
    print("Saved soil_model ->", p)
    if p: saved_paths.append(p)
else:
    print("soil_model is None; could not resolve from soil_models/soil_model_name")

if cbr is not None:
    p = save_model(cbr, "cbr_model")
    print("Saved cbr_model ->", p)
    if p: saved_paths.append(p)
else:
    print("cbr_model is None; could not resolve from cbr_models/cbr_model_name")

# List results
print("\nContents of /content/saved_model:")
for p in pathlib.Path("/content/saved_model").glob("*"):
    print(" -", p, f"({p.stat().st_size/1024/1024:.2f} MB)")


soil_model is None; could not resolve from soil_models/soil_model_name
cbr_model is None; could not resolve from cbr_models/cbr_model_name

Contents of /content/saved_model:


In [11]:
!pip -q install -U pandas scikit-learn lightgbm joblib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.


In [12]:
# ✅ Install compatible versions for Colab
!pip -q install "pandas==2.2.2" "scikit-learn==1.4.2" "lightgbm==4.3.0" "joblib==1.3.2"

# (If you see a message suggesting a runtime restart, do it once,
# then continue with Cell 2.)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 24.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml-cu12 25.6.0 requires scikit-learn>=1.5, but you have scikit-learn 1.4.2 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.4.2 which is incompatible.


In [13]:
import pandas as pd, sklearn, lightgbm
import joblib, numpy as np

print("pandas:", pd.__version__)
print("scikit-learn:", sklearn.__version__)
print("lightgbm:", lightgbm.__version__)
print("joblib:", joblib.__version__)


pandas: 2.2.2
scikit-learn: 1.6.1
lightgbm: 4.3.0
joblib: 1.5.1


In [14]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import pandas as pd

DATA_PATH = "/content/drive/MyDrive/Soil_Al/Soil_Master_with_Sources.csv"
df = pd.read_csv(DATA_PATH)

print("Rows:", len(df), "| Columns:", len(df.columns))
df.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Soil_Al/Soil_Master_with_Sources.csv'

In [17]:
DATA_PATH = "/content/drive/MyDrive/Soil_AI/Soil_Master_with_Sources.csv"

import pandas as pd
df = pd.read_csv(DATA_PATH)

print("Rows:", len(df), "| Columns:", len(df.columns))
df.head()


Rows: 181 | Columns: 9


,BatchID,Test_Type,SampleID,Soil_Type,Parameter,Value,Units,Source,Notes
0,38,Sieve,Sample_3pdf,NaN,D10(mm),0.3510775862068965,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
1,39,Sieve,Sample_5pdf,NaN,D10(mm),NaN,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
2,41,Sieve,Sample_6pdf_Sand,NaN,D10(mm),0.2134615384615384,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
3,40,Sieve,Sample_6pdf_Granite,NaN,D10(mm),NaN,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
4,38,Sieve,Sample_3pdf,NaN,D30(mm),0.8105269645608628,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)


In [2]:
# Check the unique test types
print("Unique Test Types:", df["Test_Type"].unique())

# Count how many rows per test type
print("\nRows per Test Type:")
print(df["Test_Type"].value_counts())


Unique Test Types: ['Sieve' 'Atterberg' 'Compaction' 'CBR' 'Moisture']

Rows per Test Type:
Test_Type
CBR           84
Atterberg     40
Sieve         32
Compaction    18
Moisture       7
Name: count, dtype: int64


In [3]:
import numpy as np
import pandas as pd

# Start from the stacked master already loaded as dfm
wide = dfm.pivot_table(index=["SampleID"], columns="Parameter", values="Value", aggfunc="first").reset_index()

# Coerce numerics we need
num_cols = [
    "Percent_Passing_at_0.075mm",  # #200 (0.075 mm)
    "Percent_Passing_at_0.425mm",  # #40 (0.425 mm)
    "Percent_Passing_at_2.0mm",    # #10 (2.0 mm)
    "Cu","Cc",
    "Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)"
]
for c in num_cols:
    if c in wide.columns:
        wide[c] = pd.to_numeric(wide[c], errors="coerce")

# --- Helpers ---
def uscs_from(w):
    F = w.get("Percent_Passing_at_0.075mm")
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    Cu = w.get("Cu"); Cc = w.get("Cc")
    if pd.isna(F): return None

    # Coarse vs fine
    if F < 50:
        # Coarse-grained
        # Decide sand vs gravel using 4.75 mm (#4). If not present, we’ll just say “S or G”.
        # Clean vs with fines
        fines_class = None
        if F < 5: fines_class = "clean"
        elif F > 12: fines_class = "fines"
        else: fines_class = "dual"

        # Well/poorly graded (needs Cu & Cc)
        grad = None
        if pd.notna(Cu) and pd.notna(Cc):
            # Use sand criteria (similar for gravels): Cu>6 & 1<Cc<3 (for sands Cu>6, gravels Cu>4)
            # We’ll be conservative and use: Cu>6 and 1<Cc<3 => well graded (W), else poorly graded (P)
            grad = "W" if (Cu>6 and 1<Cc<3) else "P"

        if fines_class == "clean":
            return f"S{grad}" if grad else "S"
        elif fines_class == "fines":
            if pd.isna(LL) or pd.isna(PI): return "S(M/C)"
            # A-line: PI >= 0.73*(LL-20) ⇒ clayey (C), else silty (M)
            a_line = 0.73*(LL-20)
            return "SC" if PI >= a_line else "SM"
        else:
            # 5–12% fines → dual symbols; we’ll return generic
            return "SW-SM" if grad=="W" else "SP-SM"
    else:
        # Fine-grained
        if pd.isna(LL) or pd.isna(PI): return None
        a_line = 0.73*(LL-20)
        if LL < 50:
            return "CL" if PI >= a_line else "ML"
        else:
            return "CH" if PI >= a_line else "MH"

def aashto_gi_from(w):
    # AASHTO GI = (F−35)[0.2 + 0.005(LL−40)] + 0.01(F−15)(PI−10)
    F  = w.get("Percent_Passing_at_0.075mm")   # #200
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    if any(pd.isna(x) for x in [F,LL,PI]): return None
    GI = (max(F-35,0))*(0.2 + 0.005*max(LL-40,0)) + 0.01*max(F-15,0)*max(PI-10,0)
    return round(float(GI),2)

wide["USCS_rule"] = wide.apply(uscs_from, axis=1)
wide["AASHTO_GI"] = wide.apply(aashto_gi_from, axis=1)

# Keep existing reported class if you had it; otherwise backfill with rule
if "Soil_Classification" in wide.columns:
    wide["Soil_Class_final"] = wide["Soil_Classification"]
    wide.loc[wide["Soil_Class_final"].isna() | (wide["Soil_Class_final"]==""), "Soil_Class_final"] = wide["USCS_rule"]
else:
    wide["Soil_Class_final"] = wide["USCS_rule"]

print(wide[["SampleID","USCS_rule","AASHTO_GI","Soil_Class_final"]].head(10))
wide.to_csv("labeled.csv", index=False)
print("Saved labeled.csv with rule-based USCS/AASHTO columns.")


NameError: name 'dfm' is not defined

In [4]:
import numpy as np
import pandas as pd

# --- 1) Start from the stacked master already loaded as df ---
dfm = df.copy()  # your current DataFrame variable is `df`

wide = dfm.pivot_table(
    index=["SampleID"],
    columns="Parameter",
    values="Value",
    aggfunc="first"
).reset_index()

# --- 2) Coerce numerics we need ---
num_cols = [
    "Percent_Passing_at_0.075mm",  # #200 (0.075 mm)
    "Percent_Passing_at_0.425mm",  # #40 (0.425 mm) — not strictly needed here, but ok to keep
    "Percent_Passing_at_2.0mm",    # #10 (2.0 mm) — optional
    "Cu","Cc",
    "Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)"
]
for c in num_cols:
    if c in wide.columns:
        wide[c] = pd.to_numeric(wide[c], errors="coerce")

# --- 3) Rule helpers (USCS + AASHTO GI) ---
def uscs_from(w):
    F  = w.get("Percent_Passing_at_0.075mm")      # fines (No. 200)
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    Cu = w.get("Cu"); Cc = w.get("Cc")

    # Need at least F to decide coarse vs fine
    if pd.isna(F):
        return None

    # Coarse vs fine boundary
    if F < 50:
        # Coarse-grained (S/G). We'll label as Sand (S) family for simplicity here.
        if F < 5:
            fines_class = "clean"
        elif F > 12:
            fines_class = "fines"
        else:
            fines_class = "dual"

        grad = None
        if pd.notna(Cu) and pd.notna(Cc):
            # Sand criteria (conservative): Cu > 6 and 1 < Cc < 3 => well graded
            grad = "W" if (Cu > 6 and 1 < Cc < 3) else "P"

        if fines_class == "clean":
            return f"S{grad}" if grad else "S"
        elif fines_class == "fines":
            if pd.isna(LL) or pd.isna(PI):
                return "S(M/C)"
            a_line = 0.73*(LL - 20.0)
            return "SC" if PI >= a_line else "SM"
        else:
            # 5–12% fines → dual symbol
            return "SW-SM" if grad == "W" else "SP-SM"
    else:
        # Fine-grained
        if pd.isna(LL) or pd.isna(PI):
            return None
        a_line = 0.73*(LL - 20.0)
        if LL < 50:
            return "CL" if PI >= a_line else "ML"
        else:
            return "CH" if PI >= a_line else "MH"

def aashto_gi_from(w):
    # AASHTO GI = (F−35)[0.2 + 0.005(LL−40)] + 0.01(F−15)(PI−10)
    F  = w.get("Percent_Passing_at_0.075mm")
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    if any(pd.isna(x) for x in [F, LL, PI]):
        return None
    GI = max(F-35, 0) * (0.2 + 0.005*max(LL-40, 0)) + 0.01*max(F-15, 0) * max(PI-10, 0)
    return round(float(GI), 2)

# --- 4) Apply rules ---
wide["USCS_rule"]  = wide.apply(uscs_from, axis=1)
wide["AASHTO_GI"]  = wide.apply(aashto_gi_from, axis=1)

# If your sheet had a reported class column, use that; else backfill from rule.
reported_col = None
for cand in ["Soil_Classification", "Soil_Class", "USCS_Class"]:
    if cand in wide.columns:
        reported_col = cand
        break

if reported_col:
    wide["Soil_Class_final"] = wide[reported_col]
    wide.loc[wide["Soil_Class_final"].isna() | (wide["Soil_Class_final"]==""), "Soil_Class_final"] = wide["USCS_rule"]
else:
    wide["Soil_Class_final"] = wide["USCS_rule"]

# --- 5) Save + quick look ---
print(wide[["SampleID","USCS_rule","AASHTO_GI","Soil_Class_final"]].head(10))
print("\nClass counts:\n", wide["Soil_Class_final"].value_counts(dropna=False))

wide.to_csv("labeled.csv", index=False)
print("\nSaved labeled.csv with rule-based USCS/AASHTO columns.")


Parameter                   SampleID USCS_rule AASHTO_GI  \
0            Atterberg_CSU_LongBeach      None      None   
1              Atterberg_Cyprus_Intl      None      None   
2                Atterberg_Indonesia      None      None   
3            Atterberg_Namibia_GTM7b      None      None   
4              Atterberg_UiTM_CEG454      None      None   
5             Atterberg_UiTM_ConePen      None      None   
6          Atterberg_UiTM_FullReport      None      None   
7           Atterberg_UiTM_Pahang_PL      None      None   
8            Atterberg_UiTM_ShahAlam      None      None   
9           Atterberg_UiTM_ShahAlam2      None      None   

Parameter          Soil_Class_final  
0                      Plastic Clay  
1                Non-plastic / Silt  
2            Clay (High Plasticity)  
3                              Clay  
4                 Intermediate Clay  
5          CLAY with low plasticity  
6                              None  
7             Clay (Low Plasticity)

In [21]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load the labeled dataset
data = pd.read_csv("labeled.csv")

# --- Select features and target ---
feature_cols = [
    "Liquid_Limit_LL(%)",
    "Plastic_Limit_PL(%)",
    "Plasticity_Index_PI(%)",
    "Percent_Passing_at_0.075mm",
    "Cu","Cc"
]
X = data[feature_cols].copy()
y = data["Soil_Class_final"]

# Drop rows with missing target
X = X[~y.isna()]
y = y[~y.isna()]

# Fill any missing numeric with median
X = X.fillna(X.median())

# --- Split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- Train baseline model ---
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

# --- Evaluate ---
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

# --- Save model ---
joblib.dump(rf, "soil_classifier.pkl")
joblib.dump(feature_cols, "soil_classifier_features.pkl")

print("\nModel trained and saved: soil_classifier.pkl")


ImportError: cannot import name 'check_matplotlib_support' from 'sklearn.utils' (/usr/local/lib/python3.12/dist-packages/sklearn/utils/__init__.py)

In [22]:
!pip install -U scikit-learn


  Using cached scikit_learn-1.7.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.5 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.2
    Uninstalling scikit-learn-1.4.2:
      Successfully uninstalled scikit-learn-1.4.2


In [1]:
DATA_PATH = "/content/drive/MyDrive/Soil_AI/Soil_Master_with_Sources.csv"

import pandas as pd
df = pd.read_csv(DATA_PATH)

print("Rows:", len(df), "| Columns:", len(df.columns))
df.head()


Rows: 181 | Columns: 9


,BatchID,Test_Type,SampleID,Soil_Type,Parameter,Value,Units,Source,Notes
0,38,Sieve,Sample_3pdf,NaN,D10(mm),0.3510775862068965,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
1,39,Sieve,Sample_5pdf,NaN,D10(mm),NaN,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
2,41,Sieve,Sample_6pdf_Sand,NaN,D10(mm),0.2134615384615384,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
3,40,Sieve,Sample_6pdf_Granite,NaN,D10(mm),NaN,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
4,38,Sieve,Sample_3pdf,NaN,D30(mm),0.8105269645608628,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)


In [5]:
import numpy as np
import pandas as pd

# Start from the stacked master already loaded as dfm
wide = dfm.pivot_table(index=["SampleID"], columns="Parameter", values="Value", aggfunc="first").reset_index()

# Coerce numerics we need
num_cols = [
    "Percent_Passing_at_0.075mm",  # #200 (0.075 mm)
    "Percent_Passing_at_0.425mm",  # #40 (0.425 mm)
    "Percent_Passing_at_2.0mm",    # #10 (2.0 mm)
    "Cu","Cc",
    "Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)"
]
for c in num_cols:
    if c in wide.columns:
        wide[c] = pd.to_numeric(wide[c], errors="coerce")

# --- Helpers ---
def uscs_from(w):
    F = w.get("Percent_Passing_at_0.075mm")
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    Cu = w.get("Cu"); Cc = w.get("Cc")
    if pd.isna(F): return None

    # Coarse vs fine
    if F < 50:
        # Coarse-grained
        # Decide sand vs gravel using 4.75 mm (#4). If not present, we’ll just say “S or G”.
        # Clean vs with fines
        fines_class = None
        if F < 5: fines_class = "clean"
        elif F > 12: fines_class = "fines"
        else: fines_class = "dual"

        # Well/poorly graded (needs Cu & Cc)
        grad = None
        if pd.notna(Cu) and pd.notna(Cc):
            # Use sand criteria (similar for gravels): Cu>6 & 1<Cc<3 (for sands Cu>6, gravels Cu>4)
            # We’ll be conservative and use: Cu>6 and 1<Cc<3 => well graded (W), else poorly graded (P)
            grad = "W" if (Cu>6 and 1<Cc<3) else "P"

        if fines_class == "clean":
            return f"S{grad}" if grad else "S"
        elif fines_class == "fines":
            if pd.isna(LL) or pd.isna(PI): return "S(M/C)"
            # A-line: PI >= 0.73*(LL-20) ⇒ clayey (C), else silty (M)
            a_line = 0.73*(LL-20)
            return "SC" if PI >= a_line else "SM"
        else:
            # 5–12% fines → dual symbols; we’ll return generic
            return "SW-SM" if grad=="W" else "SP-SM"
    else:
        # Fine-grained
        if pd.isna(LL) or pd.isna(PI): return None
        a_line = 0.73*(LL-20)
        if LL < 50:
            return "CL" if PI >= a_line else "ML"
        else:
            return "CH" if PI >= a_line else "MH"

def aashto_gi_from(w):
    # AASHTO GI = (F−35)[0.2 + 0.005(LL−40)] + 0.01(F−15)(PI−10)
    F  = w.get("Percent_Passing_at_0.075mm")   # #200
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    if any(pd.isna(x) for x in [F,LL,PI]): return None
    GI = (max(F-35,0))*(0.2 + 0.005*max(LL-40,0)) + 0.01*max(F-15,0)*max(PI-10,0)
    return round(float(GI),2)

wide["USCS_rule"] = wide.apply(uscs_from, axis=1)
wide["AASHTO_GI"] = wide.apply(aashto_gi_from, axis=1)

# Keep existing reported class if you had it; otherwise backfill with rule
if "Soil_Classification" in wide.columns:
    wide["Soil_Class_final"] = wide["Soil_Classification"]
    wide.loc[wide["Soil_Class_final"].isna() | (wide["Soil_Class_final"]==""), "Soil_Class_final"] = wide["USCS_rule"]
else:
    wide["Soil_Class_final"] = wide["USCS_rule"]

print(wide[["SampleID","USCS_rule","AASHTO_GI","Soil_Class_final"]].head(10))
wide.to_csv("labeled.csv", index=False)
print("Saved labeled.csv with rule-based USCS/AASHTO columns.")


Parameter                   SampleID USCS_rule AASHTO_GI  \
0            Atterberg_CSU_LongBeach      None      None   
1              Atterberg_Cyprus_Intl      None      None   
2                Atterberg_Indonesia      None      None   
3            Atterberg_Namibia_GTM7b      None      None   
4              Atterberg_UiTM_CEG454      None      None   
5             Atterberg_UiTM_ConePen      None      None   
6          Atterberg_UiTM_FullReport      None      None   
7           Atterberg_UiTM_Pahang_PL      None      None   
8            Atterberg_UiTM_ShahAlam      None      None   
9           Atterberg_UiTM_ShahAlam2      None      None   

Parameter          Soil_Class_final  
0                      Plastic Clay  
1                Non-plastic / Silt  
2            Clay (High Plasticity)  
3                              Clay  
4                 Intermediate Clay  
5          CLAY with low plasticity  
6                              None  
7             Clay (Low Plasticity)

In [6]:
# List columns we can use
print("All columns:", dfm.columns.tolist())

# What 'Percent_Passing_at_*' parameters exist after pivot?
wide_dbg = dfm.pivot_table(index="SampleID", columns="Parameter", values="Value", aggfunc="first").reset_index()
pp_cols = [c for c in wide_dbg.columns if str(c).startswith("Percent_Passing_at_")]
print("Percent_Passing_at_* columns:", pp_cols[:20])


All columns: ['BatchID', 'Test_Type', 'SampleID', 'Soil_Type', 'Parameter', 'Value', 'Units', 'Source', 'Notes']
Percent_Passing_at_* columns: []


In [7]:
import re
import numpy as np
import pandas as pd

# 1) Build a wide table from the stacked master dfm
wide = dfm.pivot_table(index=["SampleID"], columns="Parameter", values="Value", aggfunc="first").reset_index()

# 2) Coerce numerics we may use
for c in ["Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)","Cu","Cc",
          "%Fines","%Sand","%Gravel"]:
    if c in wide.columns:
        wide[c] = pd.to_numeric(wide[c], errors="coerce")

# 3) Try to find %Passing for #200 (0.075mm), #40 (0.425mm), #10 (2.0mm) if they exist,
#    otherwise backfill F200 from %Fines (typical equivalence).
pp_cols = [c for c in wide.columns if str(c).startswith("Percent_Passing_at_")]

def pick_mm(colnames, target_mm, tol):
    # choose column whose embedded size is closest to target within tolerance
    best_col, best_d = None, 1e9
    for c in colnames:
        m = re.search(r"Percent_Passing_at_([0-9.]+)\s*mm", str(c))
        if not m:
            continue
        try:
            size = float(m.group(1))
        except:
            continue
        d = abs(size - target_mm)
        if d < best_d:
            best_col, best_d = c, d
    return best_col if best_d <= tol else None

col_F200 = pick_mm(pp_cols, 0.075, tol=0.02)  # No.200
col_F40  = pick_mm(pp_cols, 0.425, tol=0.05)  # No.40
col_F10  = pick_mm(pp_cols, 2.0,   tol=0.20)  # No.10

# Create F200/F40/F10 numeric columns if found
if col_F200: wide["F200"] = pd.to_numeric(wide[col_F200], errors="coerce")
if col_F40:  wide["F40"]  = pd.to_numeric(wide[col_F40],  errors="coerce")
if col_F10:  wide["F10"]  = pd.to_numeric(wide[col_F10],  errors="coerce")

# Backfill F200 from %Fines if needed
if "F200" not in wide.columns and "%Fines" in wide.columns:
    wide["F200"] = wide["%Fines"]

# 4) USCS & AASHTO rule functions
def a_line(LL):
    return 0.73*(LL-20)

def uscs_from_row(r):
    F  = r.get("F200")
    LL = r.get("Liquid_Limit_LL(%)")
    PI = r.get("Plasticity_Index_PI(%)")
    Cu = r.get("Cu")
    Cc = r.get("Cc")
    if pd.isna(F):
        return None

    if F < 50:  # coarse-grained (S/G) — we only have sand/grading info, so default to S*
        if F < 5:
            if pd.notna(Cu) and pd.notna(Cc):
                return "SW" if (Cu>6 and 1<Cc<3) else "SP"
            return "S"   # clean sand (grading unknown)
        elif F > 12:
            if pd.isna(LL) or pd.isna(PI):
                return "S?(with fines)"
            return "SC" if PI >= a_line(LL) else "SM"
        else:  # 5–12% fines → dual symbols
            if pd.notna(Cu) and pd.notna(Cc) and (Cu>6 and 1<Cc<3):
                return "SW-SM"
            return "SP-SM"
    else:      # fine-grained
        if pd.isna(LL) or pd.isna(PI):
            return None
        return ("CL" if PI >= a_line(LL) else "ML") if LL < 50 else ("CH" if PI >= a_line(LL) else "MH")

def aashto_gi(r):
    F  = r.get("F200")
    LL = r.get("Liquid_Limit_LL(%)")
    PI = r.get("Plasticity_Index_PI(%)")
    if any(pd.isna(x) for x in [F,LL,PI]):
        return None
    GI = (max(F-35,0))*(0.2 + 0.005*max(LL-40,0)) + 0.01*max(F-15,0)*max(PI-10,0)
    return round(float(GI),2)

wide["USCS_rule"] = wide.apply(uscs_from_row, axis=1)
wide["AASHTO_GI"] = wide.apply(aashto_gi, axis=1)

# Prefer any reported class if present; else use rule
if "Soil_Classification" in wide.columns:
    wide["Soil_Class_final"] = wide["Soil_Classification"].where(
        wide["Soil_Classification"].notna() & (wide["Soil_Classification"]!=""),
        wide["USCS_rule"]
    )
else:
    wide["Soil_Class_final"] = wide["USCS_rule"]

print(wide[["SampleID","F200","Liquid_Limit_LL(%)","Plasticity_Index_PI(%)","USCS_rule","AASHTO_GI","Soil_Class_final"]].head(12))
wide.to_csv("labeled.csv", index=False)
print("✅ Saved labeled.csv with rule-based USCS & AASHTO GI.")


Parameter                   SampleID  F200  Liquid_Limit_LL(%)  \
0            Atterberg_CSU_LongBeach   NaN               48.50   
1              Atterberg_Cyprus_Intl   NaN               29.62   
2                Atterberg_Indonesia   NaN               70.00   
3            Atterberg_Namibia_GTM7b   NaN                 NaN   
4              Atterberg_UiTM_CEG454   NaN               35.80   
5             Atterberg_UiTM_ConePen   NaN               32.00   
6          Atterberg_UiTM_FullReport   NaN               48.50   
7           Atterberg_UiTM_Pahang_PL   NaN                 NaN   
8            Atterberg_UiTM_ShahAlam   NaN               26.00   
9           Atterberg_UiTM_ShahAlam2   NaN               26.00   
10                 CBR_Image_10blows   NaN                 NaN   
11                 CBR_Image_30blows   NaN                 NaN   

Parameter  Plasticity_Index_PI(%) USCS_rule AASHTO_GI  \
0                           25.60      None      None   
1                          

In [8]:
!pip -q install xgboost lightgbm >/dev/null

import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor

# Load labeled dataset
W = pd.read_csv("labeled.csv")

# ==== CLASSIFIER ====
features_cls = [c for c in ["Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)",
                            "Cu","Cc","F200"] if c in W.columns]
cls_df = W.dropna(subset=["Soil_Class_final"])[features_cls + ["Soil_Class_final"]].copy()
for c in features_cls:
    cls_df[c] = pd.to_numeric(cls_df[c], errors="coerce")
cls_df = cls_df.dropna()

Xc = cls_df[features_cls].values
le = LabelEncoder()
yc = le.fit_transform(cls_df["Soil_Class_final"].astype(str).values)

models_cls = {
    "RF": RandomForestClassifier(n_estimators=300, random_state=42, class_weight="balanced"),
    "XGB": XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=4,
                         subsample=0.9, colsample_bytree=0.9, eval_metric="mlogloss", random_state=42),
    "LGBM": LGBMClassifier(n_estimators=500, learning_rate=0.05, subsample=0.9,
                           colsample_bytree=0.9, random_state=42)
}

print("=== Soil Classifier CV Accuracy ===")
for name, model in models_cls.items():
    if len(Xc) >= 4:
        acc = cross_val_score(model, Xc, yc, cv=min(5,len(Xc)), scoring="accuracy")
        print(f"{name}: mean={acc.mean():.3f}, scores={np.round(acc,3)}")
    else:
        print(f"{name}: not enough samples")

# ==== REGRESSOR ====
features_reg = [c for c in ["Liquid_Limit_LL(%)","Plasticity_Index_PI(%)","F200","F40","F10"] if c in W.columns]
reg_df = W.dropna(subset=["AASHTO_GI"])[features_reg + ["AASHTO_GI"]].copy()
for c in features_reg:
    reg_df[c] = pd.to_numeric(reg_df[c], errors="coerce")
reg_df = reg_df.dropna()

Xr = reg_df[features_reg].values
yr = reg_df["AASHTO_GI"].values.astype(float)

models_reg = {
    "RF": RandomForestRegressor(n_estimators=400, random_state=42),
    "XGB": XGBRegressor(n_estimators=600, learning_rate=0.05, max_depth=4,
                        subsample=0.9, colsample_bytree=0.9, random_state=42),
    "LGBM": LGBMRegressor(n_estimators=600, learning_rate=0.05, subsample=0.9,
                          colsample_bytree=0.9, random_state=42)
}

print("\n=== GI Regressor CV MAE ===")
for name, model in models_reg.items():
    if len(Xr) >= 4:
        mae = -cross_val_score(model, Xr, yr, cv=min(5,len(Xr)), scoring="neg_mean_absolute_error")
        print(f"{name}: mean={mae.mean():.3f}, scores={np.round(mae,3)}")
    else:
        print(f"{name}: not enough samples")


=== Soil Classifier CV Accuracy ===
RF: not enough samples
XGB: not enough samples
LGBM: not enough samples

=== GI Regressor CV MAE ===
RF: not enough samples
XGB: not enough samples
LGBM: not enough samples


In [10]:
!pip -q install xgboost lightgbm >/dev/null

import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor

# Load labeled dataset
W = pd.read_csv("labeled.csv")

# ==== CLASSIFIER ====
features_cls = [c for c in ["Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)",
                            "Cu","Cc","F200"] if c in W.columns]
cls_df = W.dropna(subset=["Soil_Class_final"])[features_cls + ["Soil_Class_final"]].copy()
for c in features_cls:
    cls_df[c] = pd.to_numeric(cls_df[c], errors="coerce")
cls_df = cls_df.dropna()

Xc = cls_df[features_cls].values
le = LabelEncoder()
yc = le.fit_transform(cls_df["Soil_Class_final"].astype(str).values)

models_cls = {
    "RF": RandomForestClassifier(n_estimators=300, random_state=42, class_weight="balanced"),
    "XGB": XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=4,
                         subsample=0.9, colsample_bytree=0.9, eval_metric="mlogloss", random_state=42),
    "LGBM": LGBMClassifier(n_estimators=500, learning_rate=0.05, subsample=0.9,
                           colsample_bytree=0.9, random_state=42)
}

print("=== Soil Classifier CV Accuracy ===")
for name, model in models_cls.items():
    if len(Xc) >= 4:
        acc = cross_val_score(model, Xc, yc, cv=min(5,len(Xc)), scoring="accuracy")
        print(f"{name}: mean={acc.mean():.3f}, scores={np.round(acc,3)}")
    else:
        print(f"{name}: not enough samples")

# ==== REGRESSOR ====
features_reg = [c for c in ["Liquid_Limit_LL(%)","Plasticity_Index_PI(%)","F200","F40","F10"] if c in W.columns]
reg_df = W.dropna(subset=["AASHTO_GI"])[features_reg + ["AASHTO_GI"]].copy()
for c in features_reg:
    reg_df[c] = pd.to_numeric(reg_df[c], errors="coerce")
reg_df = reg_df.dropna()

Xr = reg_df[features_reg].values
yr = reg_df["AASHTO_GI"].values.astype(float)

models_reg = {
    "RF": RandomForestRegressor(n_estimators=400, random_state=42),
    "XGB": XGBRegressor(n_estimators=600, learning_rate=0.05, max_depth=4,
                        subsample=0.9, colsample_bytree=0.9, random_state=42),
    "LGBM": LGBMRegressor(n_estimators=600, learning_rate=0.05, subsample=0.9,
                          colsample_bytree=0.9, random_state=42)
}

print("\n=== GI Regressor CV MAE ===")
for name, model in models_reg.items():
    if len(Xr) >= 4:
        mae = -cross_val_score(model, Xr, yr, cv=min(5,len(Xr)), scoring="neg_mean_absolute_error")
        print(f"{name}: mean={mae.mean():.3f}, scores={np.round(mae,3)}")
    else:
        print(f"{name}: not enough samples")


=== Soil Classifier CV Accuracy ===
RF: not enough samples
XGB: not enough samples
LGBM: not enough samples

=== GI Regressor CV MAE ===
RF: not enough samples
XGB: not enough samples
LGBM: not enough samples


In [1]:
!pip -q install -U pandas scikit-learn lightgbm joblib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.


In [2]:
# ✅ Install compatible versions for Colab
!pip -q install "pandas==2.2.2" "scikit-learn==1.4.2" "lightgbm==4.3.0" "joblib==1.3.2"

# (If you see a message suggesting a runtime restart, do it once,
# then continue with Cell 2.)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 14.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml-cu12 25.6.0 requires scikit-learn>=1.5, but you have scikit-learn 1.4.2 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.4.2 which is incompatible.


In [4]:
# ✅ Install compatible versions for Colab
!pip -q install "pandas==2.2.2" "scikit-learn==1.4.2" "lightgbm==4.3.0" "joblib==1.3.2"

# (If you see a message suggesting a runtime restart, do it once,
# then continue with Cell 2.)


In [5]:
import pandas as pd, sklearn, lightgbm
import joblib, numpy as np

print("pandas:", pd.__version__)
print("scikit-learn:", sklearn.__version__)
print("lightgbm:", lightgbm.__version__)
print("joblib:", joblib.__version__)


pandas: 2.2.2
scikit-learn: 1.4.2
lightgbm: 4.3.0
joblib: 1.3.2


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
DATA_PATH = "/content/drive/MyDrive/Soil_AI/Soil_Master_with_Sources.csv"

import pandas as pd
df = pd.read_csv(DATA_PATH)

print("Rows:", len(df), "| Columns:", len(df.columns))
df.head()


Rows: 181 | Columns: 9


,BatchID,Test_Type,SampleID,Soil_Type,Parameter,Value,Units,Source,Notes
0,38,Sieve,Sample_3pdf,NaN,D10(mm),0.3510775862068965,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
1,39,Sieve,Sample_5pdf,NaN,D10(mm),NaN,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
2,41,Sieve,Sample_6pdf_Sand,NaN,D10(mm),0.2134615384615384,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
3,40,Sieve,Sample_6pdf_Granite,NaN,D10(mm),NaN,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)
4,38,Sieve,Sample_3pdf,NaN,D30(mm),0.8105269645608628,mm,Sieve – UiTM/Studocu (cleaned),Derived from uploaded lab reports (cleaned)


In [8]:
# Check the unique test types
print("Unique Test Types:", df["Test_Type"].unique())

# Count how many rows per test type
print("\nRows per Test Type:")
print(df["Test_Type"].value_counts())


Unique Test Types: ['Sieve' 'Atterberg' 'Compaction' 'CBR' 'Moisture']

Rows per Test Type:
Test_Type
CBR           84
Atterberg     40
Sieve         32
Compaction    18
Moisture       7
Name: count, dtype: int64


In [9]:
import numpy as np
import pandas as pd

# --- 1) Start from the stacked master already loaded as df ---
dfm = df.copy()  # your current DataFrame variable is `df`

wide = dfm.pivot_table(
    index=["SampleID"],
    columns="Parameter",
    values="Value",
    aggfunc="first"
).reset_index()

# --- 2) Coerce numerics we need ---
num_cols = [
    "Percent_Passing_at_0.075mm",  # #200 (0.075 mm)
    "Percent_Passing_at_0.425mm",  # #40 (0.425 mm) — not strictly needed here, but ok to keep
    "Percent_Passing_at_2.0mm",    # #10 (2.0 mm) — optional
    "Cu","Cc",
    "Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)"
]
for c in num_cols:
    if c in wide.columns:
        wide[c] = pd.to_numeric(wide[c], errors="coerce")

# --- 3) Rule helpers (USCS + AASHTO GI) ---
def uscs_from(w):
    F  = w.get("Percent_Passing_at_0.075mm")      # fines (No. 200)
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    Cu = w.get("Cu"); Cc = w.get("Cc")

    # Need at least F to decide coarse vs fine
    if pd.isna(F):
        return None

    # Coarse vs fine boundary
    if F < 50:
        # Coarse-grained (S/G). We'll label as Sand (S) family for simplicity here.
        if F < 5:
            fines_class = "clean"
        elif F > 12:
            fines_class = "fines"
        else:
            fines_class = "dual"

        grad = None
        if pd.notna(Cu) and pd.notna(Cc):
            # Sand criteria (conservative): Cu > 6 and 1 < Cc < 3 => well graded
            grad = "W" if (Cu > 6 and 1 < Cc < 3) else "P"

        if fines_class == "clean":
            return f"S{grad}" if grad else "S"
        elif fines_class == "fines":
            if pd.isna(LL) or pd.isna(PI):
                return "S(M/C)"
            a_line = 0.73*(LL - 20.0)
            return "SC" if PI >= a_line else "SM"
        else:
            # 5–12% fines → dual symbol
            return "SW-SM" if grad == "W" else "SP-SM"
    else:
        # Fine-grained
        if pd.isna(LL) or pd.isna(PI):
            return None
        a_line = 0.73*(LL - 20.0)
        if LL < 50:
            return "CL" if PI >= a_line else "ML"
        else:
            return "CH" if PI >= a_line else "MH"

def aashto_gi_from(w):
    # AASHTO GI = (F−35)[0.2 + 0.005(LL−40)] + 0.01(F−15)(PI−10)
    F  = w.get("Percent_Passing_at_0.075mm")
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    if any(pd.isna(x) for x in [F, LL, PI]):
        return None
    GI = max(F-35, 0) * (0.2 + 0.005*max(LL-40, 0)) + 0.01*max(F-15, 0) * max(PI-10, 0)
    return round(float(GI), 2)

# --- 4) Apply rules ---
wide["USCS_rule"]  = wide.apply(uscs_from, axis=1)
wide["AASHTO_GI"]  = wide.apply(aashto_gi_from, axis=1)

# If your sheet had a reported class column, use that; else backfill from rule.
reported_col = None
for cand in ["Soil_Classification", "Soil_Class", "USCS_Class"]:
    if cand in wide.columns:
        reported_col = cand
        break

if reported_col:
    wide["Soil_Class_final"] = wide[reported_col]
    wide.loc[wide["Soil_Class_final"].isna() | (wide["Soil_Class_final"]==""), "Soil_Class_final"] = wide["USCS_rule"]
else:
    wide["Soil_Class_final"] = wide["USCS_rule"]

# --- 5) Save + quick look ---
print(wide[["SampleID","USCS_rule","AASHTO_GI","Soil_Class_final"]].head(10))
print("\nClass counts:\n", wide["Soil_Class_final"].value_counts(dropna=False))

wide.to_csv("labeled.csv", index=False)
print("\nSaved labeled.csv with rule-based USCS/AASHTO columns.")


Parameter                   SampleID USCS_rule AASHTO_GI  \
0            Atterberg_CSU_LongBeach      None      None   
1              Atterberg_Cyprus_Intl      None      None   
2                Atterberg_Indonesia      None      None   
3            Atterberg_Namibia_GTM7b      None      None   
4              Atterberg_UiTM_CEG454      None      None   
5             Atterberg_UiTM_ConePen      None      None   
6          Atterberg_UiTM_FullReport      None      None   
7           Atterberg_UiTM_Pahang_PL      None      None   
8            Atterberg_UiTM_ShahAlam      None      None   
9           Atterberg_UiTM_ShahAlam2      None      None   

Parameter          Soil_Class_final  
0                      Plastic Clay  
1                Non-plastic / Silt  
2            Clay (High Plasticity)  
3                              Clay  
4                 Intermediate Clay  
5          CLAY with low plasticity  
6                              None  
7             Clay (Low Plasticity)

In [10]:
!pip install -U scikit-learn


  Using cached scikit_learn-1.7.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.5 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.2
    Uninstalling scikit-learn-1.4.2:
      Successfully uninstalled scikit-learn-1.4.2


In [11]:
import numpy as np
import pandas as pd

# Start from the stacked master already loaded as dfm
wide = dfm.pivot_table(index=["SampleID"], columns="Parameter", values="Value", aggfunc="first").reset_index()

# Coerce numerics we need
num_cols = [
    "Percent_Passing_at_0.075mm",  # #200 (0.075 mm)
    "Percent_Passing_at_0.425mm",  # #40 (0.425 mm)
    "Percent_Passing_at_2.0mm",    # #10 (2.0 mm)
    "Cu","Cc",
    "Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)"
]
for c in num_cols:
    if c in wide.columns:
        wide[c] = pd.to_numeric(wide[c], errors="coerce")

# --- Helpers ---
def uscs_from(w):
    F = w.get("Percent_Passing_at_0.075mm")
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    Cu = w.get("Cu"); Cc = w.get("Cc")
    if pd.isna(F): return None

    # Coarse vs fine
    if F < 50:
        # Coarse-grained
        # Decide sand vs gravel using 4.75 mm (#4). If not present, we’ll just say “S or G”.
        # Clean vs with fines
        fines_class = None
        if F < 5: fines_class = "clean"
        elif F > 12: fines_class = "fines"
        else: fines_class = "dual"

        # Well/poorly graded (needs Cu & Cc)
        grad = None
        if pd.notna(Cu) and pd.notna(Cc):
            # Use sand criteria (similar for gravels): Cu>6 & 1<Cc<3 (for sands Cu>6, gravels Cu>4)
            # We’ll be conservative and use: Cu>6 and 1<Cc<3 => well graded (W), else poorly graded (P)
            grad = "W" if (Cu>6 and 1<Cc<3) else "P"

        if fines_class == "clean":
            return f"S{grad}" if grad else "S"
        elif fines_class == "fines":
            if pd.isna(LL) or pd.isna(PI): return "S(M/C)"
            # A-line: PI >= 0.73*(LL-20) ⇒ clayey (C), else silty (M)
            a_line = 0.73*(LL-20)
            return "SC" if PI >= a_line else "SM"
        else:
            # 5–12% fines → dual symbols; we’ll return generic
            return "SW-SM" if grad=="W" else "SP-SM"
    else:
        # Fine-grained
        if pd.isna(LL) or pd.isna(PI): return None
        a_line = 0.73*(LL-20)
        if LL < 50:
            return "CL" if PI >= a_line else "ML"
        else:
            return "CH" if PI >= a_line else "MH"

def aashto_gi_from(w):
    # AASHTO GI = (F−35)[0.2 + 0.005(LL−40)] + 0.01(F−15)(PI−10)
    F  = w.get("Percent_Passing_at_0.075mm")   # #200
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    if any(pd.isna(x) for x in [F,LL,PI]): return None
    GI = (max(F-35,0))*(0.2 + 0.005*max(LL-40,0)) + 0.01*max(F-15,0)*max(PI-10,0)
    return round(float(GI),2)

wide["USCS_rule"] = wide.apply(uscs_from, axis=1)
wide["AASHTO_GI"] = wide.apply(aashto_gi_from, axis=1)

# Keep existing reported class if you had it; otherwise backfill with rule
if "Soil_Classification" in wide.columns:
    wide["Soil_Class_final"] = wide["Soil_Classification"]
    wide.loc[wide["Soil_Class_final"].isna() | (wide["Soil_Class_final"]==""), "Soil_Class_final"] = wide["USCS_rule"]
else:
    wide["Soil_Class_final"] = wide["USCS_rule"]

print(wide[["SampleID","USCS_rule","AASHTO_GI","Soil_Class_final"]].head(10))
wide.to_csv("labeled.csv", index=False)
print("Saved labeled.csv with rule-based USCS/AASHTO columns.")


Parameter                   SampleID USCS_rule AASHTO_GI  \
0            Atterberg_CSU_LongBeach      None      None   
1              Atterberg_Cyprus_Intl      None      None   
2                Atterberg_Indonesia      None      None   
3            Atterberg_Namibia_GTM7b      None      None   
4              Atterberg_UiTM_CEG454      None      None   
5             Atterberg_UiTM_ConePen      None      None   
6          Atterberg_UiTM_FullReport      None      None   
7           Atterberg_UiTM_Pahang_PL      None      None   
8            Atterberg_UiTM_ShahAlam      None      None   
9           Atterberg_UiTM_ShahAlam2      None      None   

Parameter          Soil_Class_final  
0                      Plastic Clay  
1                Non-plastic / Silt  
2            Clay (High Plasticity)  
3                              Clay  
4                 Intermediate Clay  
5          CLAY with low plasticity  
6                              None  
7             Clay (Low Plasticity)

In [12]:
import numpy as np
import pandas as pd

# Start from the stacked master already loaded as dfm
wide = dfm.pivot_table(index=["SampleID"], columns="Parameter", values="Value", aggfunc="first").reset_index()

# Coerce numerics we need
num_cols = [
    "Percent_Passing_at_0.075mm",  # #200 (0.075 mm)
    "Percent_Passing_at_0.425mm",  # #40 (0.425 mm)
    "Percent_Passing_at_2.0mm",    # #10 (2.0 mm)
    "Cu","Cc",
    "Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)"
]
for c in num_cols:
    if c in wide.columns:
        wide[c] = pd.to_numeric(wide[c], errors="coerce")

# --- Helpers ---
def uscs_from(w):
    F = w.get("Percent_Passing_at_0.075mm")
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    Cu = w.get("Cu"); Cc = w.get("Cc")
    if pd.isna(F): return None

    # Coarse vs fine
    if F < 50:
        # Coarse-grained
        # Decide sand vs gravel using 4.75 mm (#4). If not present, we’ll just say “S or G”.
        # Clean vs with fines
        fines_class = None
        if F < 5: fines_class = "clean"
        elif F > 12: fines_class = "fines"
        else: fines_class = "dual"

        # Well/poorly graded (needs Cu & Cc)
        grad = None
        if pd.notna(Cu) and pd.notna(Cc):
            # Use sand criteria (similar for gravels): Cu>6 & 1<Cc<3 (for sands Cu>6, gravels Cu>4)
            # We’ll be conservative and use: Cu>6 and 1<Cc<3 => well graded (W), else poorly graded (P)
            grad = "W" if (Cu>6 and 1<Cc<3) else "P"

        if fines_class == "clean":
            return f"S{grad}" if grad else "S"
        elif fines_class == "fines":
            if pd.isna(LL) or pd.isna(PI): return "S(M/C)"
            # A-line: PI >= 0.73*(LL-20) ⇒ clayey (C), else silty (M)
            a_line = 0.73*(LL-20)
            return "SC" if PI >= a_line else "SM"
        else:
            # 5–12% fines → dual symbols; we’ll return generic
            return "SW-SM" if grad=="W" else "SP-SM"
    else:
        # Fine-grained
        if pd.isna(LL) or pd.isna(PI): return None
        a_line = 0.73*(LL-20)
        if LL < 50:
            return "CL" if PI >= a_line else "ML"
        else:
            return "CH" if PI >= a_line else "MH"

def aashto_gi_from(w):
    # AASHTO GI = (F−35)[0.2 + 0.005(LL−40)] + 0.01(F−15)(PI−10)
    F  = w.get("Percent_Passing_at_0.075mm")   # #200
    LL = w.get("Liquid_Limit_LL(%)")
    PI = w.get("Plasticity_Index_PI(%)")
    if any(pd.isna(x) for x in [F,LL,PI]): return None
    GI = (max(F-35,0))*(0.2 + 0.005*max(LL-40,0)) + 0.01*max(F-15,0)*max(PI-10,0)
    return round(float(GI),2)

wide["USCS_rule"] = wide.apply(uscs_from, axis=1)
wide["AASHTO_GI"] = wide.apply(aashto_gi_from, axis=1)

# Keep existing reported class if you had it; otherwise backfill with rule
if "Soil_Classification" in wide.columns:
    wide["Soil_Class_final"] = wide["Soil_Classification"]
    wide.loc[wide["Soil_Class_final"].isna() | (wide["Soil_Class_final"]==""), "Soil_Class_final"] = wide["USCS_rule"]
else:
    wide["Soil_Class_final"] = wide["USCS_rule"]

print(wide[["SampleID","USCS_rule","AASHTO_GI","Soil_Class_final"]].head(10))
wide.to_csv("labeled.csv", index=False)
print("Saved labeled.csv with rule-based USCS/AASHTO columns.")


Parameter                   SampleID USCS_rule AASHTO_GI  \
0            Atterberg_CSU_LongBeach      None      None   
1              Atterberg_Cyprus_Intl      None      None   
2                Atterberg_Indonesia      None      None   
3            Atterberg_Namibia_GTM7b      None      None   
4              Atterberg_UiTM_CEG454      None      None   
5             Atterberg_UiTM_ConePen      None      None   
6          Atterberg_UiTM_FullReport      None      None   
7           Atterberg_UiTM_Pahang_PL      None      None   
8            Atterberg_UiTM_ShahAlam      None      None   
9           Atterberg_UiTM_ShahAlam2      None      None   

Parameter          Soil_Class_final  
0                      Plastic Clay  
1                Non-plastic / Silt  
2            Clay (High Plasticity)  
3                              Clay  
4                 Intermediate Clay  
5          CLAY with low plasticity  
6                              None  
7             Clay (Low Plasticity)

In [13]:
import re
import numpy as np
import pandas as pd

# 1) Build a wide table from the stacked master dfm
wide = dfm.pivot_table(index=["SampleID"], columns="Parameter", values="Value", aggfunc="first").reset_index()

# 2) Coerce numerics we may use
for c in ["Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)","Cu","Cc",
          "%Fines","%Sand","%Gravel"]:
    if c in wide.columns:
        wide[c] = pd.to_numeric(wide[c], errors="coerce")

# 3) Try to find %Passing for #200 (0.075mm), #40 (0.425mm), #10 (2.0mm) if they exist,
#    otherwise backfill F200 from %Fines (typical equivalence).
pp_cols = [c for c in wide.columns if str(c).startswith("Percent_Passing_at_")]

def pick_mm(colnames, target_mm, tol):
    # choose column whose embedded size is closest to target within tolerance
    best_col, best_d = None, 1e9
    for c in colnames:
        m = re.search(r"Percent_Passing_at_([0-9.]+)\s*mm", str(c))
        if not m:
            continue
        try:
            size = float(m.group(1))
        except:
            continue
        d = abs(size - target_mm)
        if d < best_d:
            best_col, best_d = c, d
    return best_col if best_d <= tol else None

col_F200 = pick_mm(pp_cols, 0.075, tol=0.02)  # No.200
col_F40  = pick_mm(pp_cols, 0.425, tol=0.05)  # No.40
col_F10  = pick_mm(pp_cols, 2.0,   tol=0.20)  # No.10

# Create F200/F40/F10 numeric columns if found
if col_F200: wide["F200"] = pd.to_numeric(wide[col_F200], errors="coerce")
if col_F40:  wide["F40"]  = pd.to_numeric(wide[col_F40],  errors="coerce")
if col_F10:  wide["F10"]  = pd.to_numeric(wide[col_F10],  errors="coerce")

# Backfill F200 from %Fines if needed
if "F200" not in wide.columns and "%Fines" in wide.columns:
    wide["F200"] = wide["%Fines"]

# 4) USCS & AASHTO rule functions
def a_line(LL):
    return 0.73*(LL-20)

def uscs_from_row(r):
    F  = r.get("F200")
    LL = r.get("Liquid_Limit_LL(%)")
    PI = r.get("Plasticity_Index_PI(%)")
    Cu = r.get("Cu")
    Cc = r.get("Cc")
    if pd.isna(F):
        return None

    if F < 50:  # coarse-grained (S/G) — we only have sand/grading info, so default to S*
        if F < 5:
            if pd.notna(Cu) and pd.notna(Cc):
                return "SW" if (Cu>6 and 1<Cc<3) else "SP"
            return "S"   # clean sand (grading unknown)
        elif F > 12:
            if pd.isna(LL) or pd.isna(PI):
                return "S?(with fines)"
            return "SC" if PI >= a_line(LL) else "SM"
        else:  # 5–12% fines → dual symbols
            if pd.notna(Cu) and pd.notna(Cc) and (Cu>6 and 1<Cc<3):
                return "SW-SM"
            return "SP-SM"
    else:      # fine-grained
        if pd.isna(LL) or pd.isna(PI):
            return None
        return ("CL" if PI >= a_line(LL) else "ML") if LL < 50 else ("CH" if PI >= a_line(LL) else "MH")

def aashto_gi(r):
    F  = r.get("F200")
    LL = r.get("Liquid_Limit_LL(%)")
    PI = r.get("Plasticity_Index_PI(%)")
    if any(pd.isna(x) for x in [F,LL,PI]):
        return None
    GI = (max(F-35,0))*(0.2 + 0.005*max(LL-40,0)) + 0.01*max(F-15,0)*max(PI-10,0)
    return round(float(GI),2)

wide["USCS_rule"] = wide.apply(uscs_from_row, axis=1)
wide["AASHTO_GI"] = wide.apply(aashto_gi, axis=1)

# Prefer any reported class if present; else use rule
if "Soil_Classification" in wide.columns:
    wide["Soil_Class_final"] = wide["Soil_Classification"].where(
        wide["Soil_Classification"].notna() & (wide["Soil_Classification"]!=""),
        wide["USCS_rule"]
    )
else:
    wide["Soil_Class_final"] = wide["USCS_rule"]

print(wide[["SampleID","F200","Liquid_Limit_LL(%)","Plasticity_Index_PI(%)","USCS_rule","AASHTO_GI","Soil_Class_final"]].head(12))
wide.to_csv("labeled.csv", index=False)
print("✅ Saved labeled.csv with rule-based USCS & AASHTO GI.")


Parameter                   SampleID  F200  Liquid_Limit_LL(%)  \
0            Atterberg_CSU_LongBeach   NaN               48.50   
1              Atterberg_Cyprus_Intl   NaN               29.62   
2                Atterberg_Indonesia   NaN               70.00   
3            Atterberg_Namibia_GTM7b   NaN                 NaN   
4              Atterberg_UiTM_CEG454   NaN               35.80   
5             Atterberg_UiTM_ConePen   NaN               32.00   
6          Atterberg_UiTM_FullReport   NaN               48.50   
7           Atterberg_UiTM_Pahang_PL   NaN                 NaN   
8            Atterberg_UiTM_ShahAlam   NaN               26.00   
9           Atterberg_UiTM_ShahAlam2   NaN               26.00   
10                 CBR_Image_10blows   NaN                 NaN   
11                 CBR_Image_30blows   NaN                 NaN   

Parameter  Plasticity_Index_PI(%) USCS_rule AASHTO_GI  \
0                           25.60      None      None   
1                          

In [14]:
!pip -q install xgboost lightgbm >/dev/null

import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor

# Load labeled dataset
W = pd.read_csv("labeled.csv")

# ==== CLASSIFIER ====
features_cls = [c for c in ["Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)",
                            "Cu","Cc","F200"] if c in W.columns]
cls_df = W.dropna(subset=["Soil_Class_final"])[features_cls + ["Soil_Class_final"]].copy()
for c in features_cls:
    cls_df[c] = pd.to_numeric(cls_df[c], errors="coerce")
cls_df = cls_df.dropna()

Xc = cls_df[features_cls].values
le = LabelEncoder()
yc = le.fit_transform(cls_df["Soil_Class_final"].astype(str).values)

models_cls = {
    "RF": RandomForestClassifier(n_estimators=300, random_state=42, class_weight="balanced"),
    "XGB": XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=4,
                         subsample=0.9, colsample_bytree=0.9, eval_metric="mlogloss", random_state=42),
    "LGBM": LGBMClassifier(n_estimators=500, learning_rate=0.05, subsample=0.9,
                           colsample_bytree=0.9, random_state=42)
}

print("=== Soil Classifier CV Accuracy ===")
for name, model in models_cls.items():
    if len(Xc) >= 4:
        acc = cross_val_score(model, Xc, yc, cv=min(5,len(Xc)), scoring="accuracy")
        print(f"{name}: mean={acc.mean():.3f}, scores={np.round(acc,3)}")
    else:
        print(f"{name}: not enough samples")

# ==== REGRESSOR ====
features_reg = [c for c in ["Liquid_Limit_LL(%)","Plasticity_Index_PI(%)","F200","F40","F10"] if c in W.columns]
reg_df = W.dropna(subset=["AASHTO_GI"])[features_reg + ["AASHTO_GI"]].copy()
for c in features_reg:
    reg_df[c] = pd.to_numeric(reg_df[c], errors="coerce")
reg_df = reg_df.dropna()

Xr = reg_df[features_reg].values
yr = reg_df["AASHTO_GI"].values.astype(float)

models_reg = {
    "RF": RandomForestRegressor(n_estimators=400, random_state=42),
    "XGB": XGBRegressor(n_estimators=600, learning_rate=0.05, max_depth=4,
                        subsample=0.9, colsample_bytree=0.9, random_state=42),
    "LGBM": LGBMRegressor(n_estimators=600, learning_rate=0.05, subsample=0.9,
                          colsample_bytree=0.9, random_state=42)
}

print("\n=== GI Regressor CV MAE ===")
for name, model in models_reg.items():
    if len(Xr) >= 4:
        mae = -cross_val_score(model, Xr, yr, cv=min(5,len(Xr)), scoring="neg_mean_absolute_error")
        print(f"{name}: mean={mae.mean():.3f}, scores={np.round(mae,3)}")
    else:
        print(f"{name}: not enough samples")


ImportError: cannot import name '_check_n_features' from 'sklearn.utils.validation' (/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py)

In [15]:
!pip -q install xgboost lightgbm >/dev/null

import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor

# Load labeled dataset
W = pd.read_csv("labeled.csv")

# ==== CLASSIFIER ====
features_cls = [c for c in ["Liquid_Limit_LL(%)","Plastic_Limit_PL(%)","Plasticity_Index_PI(%)",
                            "Cu","Cc","F200"] if c in W.columns]
cls_df = W.dropna(subset=["Soil_Class_final"])[features_cls + ["Soil_Class_final"]].copy()
for c in features_cls:
    cls_df[c] = pd.to_numeric(cls_df[c], errors="coerce")
cls_df = cls_df.dropna()

Xc = cls_df[features_cls].values
le = LabelEncoder()
yc = le.fit_transform(cls_df["Soil_Class_final"].astype(str).values)

models_cls = {
    "RF": RandomForestClassifier(n_estimators=300, random_state=42, class_weight="balanced"),
    "XGB": XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=4,
                         subsample=0.9, colsample_bytree=0.9, eval_metric="mlogloss", random_state=42),
    "LGBM": LGBMClassifier(n_estimators=500, learning_rate=0.05, subsample=0.9,
                           colsample_bytree=0.9, random_state=42)
}

print("=== Soil Classifier CV Accuracy ===")
for name, model in models_cls.items():
    if len(Xc) >= 4:
        acc = cross_val_score(model, Xc, yc, cv=min(5,len(Xc)), scoring="accuracy")
        print(f"{name}: mean={acc.mean():.3f}, scores={np.round(acc,3)}")
    else:
        print(f"{name}: not enough samples")

# ==== REGRESSOR ====
features_reg = [c for c in ["Liquid_Limit_LL(%)","Plasticity_Index_PI(%)","F200","F40","F10"] if c in W.columns]
reg_df = W.dropna(subset=["AASHTO_GI"])[features_reg + ["AASHTO_GI"]].copy()
for c in features_reg:
    reg_df[c] = pd.to_numeric(reg_df[c], errors="coerce")
reg_df = reg_df.dropna()

Xr = reg_df[features_reg].values
yr = reg_df["AASHTO_GI"].values.astype(float)

models_reg = {
    "RF": RandomForestRegressor(n_estimators=400, random_state=42),
    "XGB": XGBRegressor(n_estimators=600, learning_rate=0.05, max_depth=4,
                        subsample=0.9, colsample_bytree=0.9, random_state=42),
    "LGBM": LGBMRegressor(n_estimators=600, learning_rate=0.05, subsample=0.9,
                          colsample_bytree=0.9, random_state=42)
}

print("\n=== GI Regressor CV MAE ===")
for name, model in models_reg.items():
    if len(Xr) >= 4:
        mae = -cross_val_score(model, Xr, yr, cv=min(5,len(Xr)), scoring="neg_mean_absolute_error")
        print(f"{name}: mean={mae.mean():.3f}, scores={np.round(mae,3)}")
    else:
        print(f"{name}: not enough samples")


ImportError: cannot import name '_check_n_features' from 'sklearn.utils.validation' (/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py)